# Fifth order Dixon functions as Kronecker theta functions derived from the Frobenius-Stickelberger determinant

This notebook explores fith order Dixon functions. Their name references Dixon functions because the well known Dixon functions are second order fully degenerate Kronecker theta functions, where degeneracy means taking the limit where all modes become the same in the Frobenius-Stickelberger determinant formula for Weierstrass sigma functions. Their name references fith order because in this example they are dervied from the Frobenius-Stickelberger determinant formula involving a product of five modes. This is a very interesting case because it is the first order at which two different Hamiltonian systems can be built from the same functional identity (conservation law). If the Hamiltonian is expressed as a polynomial in the mode and its conjugate, as is typically done at lower orders, then, unlike at lower orders, at this order that will build a dynamic system solved by hyperelliptic functions of genus 2. However, it is still possible to build a Hamiltonian that is solved by elliptic functions of genus 1 (fully degenerate Kronecker theta functions) but the Hamiltonian is no longer a polynomial in the modes but rather a rational function of them.

In [1]:
from sympy import *
(x, y, X, Y, Z, B, C, n, m, l, j, q, N, M, K, k, z, s, p1, p2, p3, g2, g3, rho,
 z0, w1, w2, w3, e1, e2, e3, Delta, 
 omega1, omega2, omega3, eta1, eta2, eta3, d1, d2, d3, t, t0) = symbols(
    '''x, y, X, Y, Z, B, C, n, m, l, j, q, N, M, K, k, z, s, p1, p2, p3, g2, g3, rho,
    z0, w1, w2, w3, e1, e2, e3, Delta, 
    omega1, omega2, omega3, eta1, eta2, eta3, d1, d2, d3, t, t0'''
)
alpha, delta, nu, theta, Delta = symbols('alpha, delta, nu, theta, Delta')
pw = Function('pw') # Weierstrass P function
pwp = Function('pwp') # Derivative of Weierstrass P function
zw = Function('zw') # Weierstrass Zeta function
sigma = Function('sigma') # Weierstrass Sigma function
rho = Function('rho')
f = Function('f')
h = Function('h')
s = Function('s')
u = Function('u')
v = Function('v')
r = Function('r')
chi = Function('chi')
P = Function('P') # Polynomial
Q = Function('Q') # Polynomial
R = Function('R') # Polynomial
T = Function('T') # Polynomial
phi = Function('phi')
Phi = Function('Phi')
phi1 = Function('phi1')
phi2 = Function('phi2')
phi3 = Function('phi3')
A = IndexedBase('A')
W1 = Function('W1')
W2 = Function('W2')
W3 = Function('W3')
W4 = Function('W4')
W5 = Function('W5')
W6 = Function('W6')
Det = Function("Det")

t0 = Symbol('t0')

kappa = IndexedBase('kappa')
beta = IndexedBase('beta')
mu = IndexedBase('mu')
xi = IndexedBase('xi')
a = IndexedBase('a')
b = IndexedBase('b')
c = IndexedBase('c')
d = IndexedBase('d')
p = IndexedBase('p')
G = IndexedBase('G')
psi = IndexedBase('psi')
upsilon = IndexedBase('upsilon')
epsilon = IndexedBase('epsilon')
WPdet = Function('WPdet')
U = Function('U')
V = Function('V')
W = Function('W')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# kth order derivatives of Weierstrass P
from wpk import wpk, run_tests

# The package containing mpmath expressions for Weierstrass elliptic functions
from numerical_evaluation.weierstrass_modified import Weierstrass
we = Weierstrass()
from mpmath import exp as mpexp

# Nuemric solutions to diff eqs
from numpy import linspace, absolute, angle, square, real, imag, conj, array as arraynp, concatenate
from numpy import vectorize as np_vectorize # not to get confused with vectorise in other packages
import scipy.integrate
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

In [2]:
run_tests()

True
True
True
True
True
True


## Weierstrass elliptic function identities

In [3]:
pwp_sqrd = Eq(pwp(z,g2,g3)**2, 4*pw(z,g2,g3)**3 - g2*pw(z,g2,g3) - g3)

pw_xy_addition = Eq(pw(x+y,g2,g3), 
   ((pwp(x, g2, g3) - pwp(y, g2, g3))**2/(pw(x, g2, g3) - pw(y, g2, g3))**2)/4 - pw(x, g2, g3) - pw(y, g2, g3)
)
p_duplication_a = Eq(pw(2*z,g2,g3), -2*pw(z,g2,g3) + ((diff(pw(z,g2,g3),z,2)/diff(pw(z,g2,g3),z,1))**2)/4)
p_duplication_b = Eq(pw(2*z, g2, g3), (-g2/2 + 6*pw(z, g2, g3)**2)**2/(4*pwp(z, g2, g3)**2) - 2*pw(z, g2, g3))
p_duplication_c = p_duplication_b.subs(pwp(z,g2,g3)**2, 4*pw(z,g2,g3)**3 - g2*pw(z,g2,g3) - g3)

sigma_p_identity = Eq(
    pw(y, g2, g3) - pw(x, g2, g3),
    sigma(x + y, g2, g3) * sigma(x - y, g2, g3) / (sigma(x, g2, g3) ** 2 * sigma(y, g2, g3) ** 2) 
)

pw_to_zw_identity = Eq(
    (pwp(x,g2,g3) - pwp(y,g2,g3))/(pw(x,g2,g3) - pw(y,g2,g3))/2,
    zw(x + y,g2, g3) - zw(x,g2, g3) - zw(y,g2, g3)
)

# 23.10.9 https://dlmf.nist.gov/23.10#E9
zw2z_2zwz = Eq(zw(2*z,g2,g3), 2*zw(z,g2,g3) + diff(zw(z,g2,g3),z,3)/diff(zw(z,g2,g3),z,2)/2)
dzw_2 = Eq(diff(zw(z,g2,g3),z,2), -pwp(z,g2,g3))
dzw_3 = Eq(diff(zw(z,g2,g3),z,3), -diff(pwp(z,g2,g3),z))
pwp_def = Eq(pwp(z,g2,g3)**2, 4*pw(z,g2,g3)**3 - g2*pw(z,g2,g3) - g3)
pwp2_def = Eq(diff(pwp_def.lhs,z)/pwp(z,g2,g3)/2, 
              diff(pwp_def.rhs,z).subs(diff(pw(z,g2,g3),z),pwp(z,g2,g3))/pwp(z,g2,g3)/2).expand()
zw2z_2zwz_pw_pwp = zw2z_2zwz.subs([dzw_3.args, dzw_2.args]).subs(*pwp2_def.args)

pwp_sigma_dbl_ratio = Eq(pwp(z,g2,g3), - sigma(2*z,g2,g3)/sigma(z,g2,g3)**4)
d_zw_pw = Eq(diff(zw(z,g2,g3),z), -pw(z,g2,g3))
d_log_sigma_zw = Eq(Derivative(ln(sigma(z,g2,g3)),z), zw(z,g2,g3))
d_sigma_zw = Eq(diff(sigma(z,g2,g3),z), solve(d_log_sigma_zw.doit(), diff(sigma(z,g2,g3),z))[0])
d_sigma_zw_2 = Eq(diff(d_sigma_zw.lhs,z), diff(d_sigma_zw.rhs,z).subs([d_sigma_zw.args, d_zw_pw.args]))
pw_diff_at_zero = Eq(Limit(Derivative(pw(z, g2, g3), (z, j)), z, 0, dir='+'),
 (-1)**j*z**(-j - 2)*factorial(j + 1) - 
                     (-1)**j*((-1)**j/2 + Rational(1,2))*RisingFactorial(-j - 1, j + 1)*G[j + 2])


expansion_z_wp = Eq(pw(z,g2,g3), 1/z**2 + Sum((2*n+1)*G[2*n+2]*z**(2*n),(n,1,oo)))
one_over_sigma_z_big_o = Eq(1/sigma(z,g2,g3), (1/z*(1 - (- g2*z**4/10 - 6*g3/7*z**6 + O(z**8)))).expand())
sigma_expansion_g_2_3 = Eq(sigma(z, g2, g3),
    85860*g2**2*g3**2*z**21/7 + 171*g2**2*g3*z**15/10 - g2**2*z**9/4 + 
    9936*g2*g3**2*z**17/17 - 18*g2*g3*z**11/11 - g2*z**5/10 - 
    216*g3**2*z**13/13 - 6*g3*z**7/7 + z
)

quasi_period = Eq(sigma(2*m*omega3 + 2*n*omega1 + z, g2, g3),
 (-1)**(m*n + m + n)*sigma(z, g2, g3)*
                  exp(2*eta1*m*n*omega3 + 2*eta1*n**2*omega1 + 2*eta1*n*z + 
                      2*eta3*m**2*omega3 + 2*eta3*m*n*omega1 + 2*eta3*m*z)
                 )



taylor_series = Eq(f(x), Sum(Derivative(f(x),(x,n)).subs(x,a)/factorial(n) * (x - a)**n, (n,0,N)))
taylor_series_b = taylor_series.subs(x,x+a)




pwp_sqrd
pw_xy_addition
p_duplication_a
p_duplication_b
p_duplication_c
sigma_p_identity
pw_to_zw_identity
zw2z_2zwz_pw_pwp
pwp_sigma_dbl_ratio
d_zw_pw
d_log_sigma_zw
d_sigma_zw
d_sigma_zw_2

pw_diff_at_zero
expansion_z_wp
one_over_sigma_z_big_o
sigma_expansion_g_2_3
quasi_period
taylor_series_b

Eq(pwp(z, g2, g3)**2, -g2*pw(z, g2, g3) - g3 + 4*pw(z, g2, g3)**3)

Eq(pw(x + y, g2, g3), -pw(x, g2, g3) - pw(y, g2, g3) + (pwp(x, g2, g3) - pwp(y, g2, g3))**2/(4*(pw(x, g2, g3) - pw(y, g2, g3))**2))

Eq(pw(2*z, g2, g3), -2*pw(z, g2, g3) + Derivative(pw(z, g2, g3), (z, 2))**2/(4*Derivative(pw(z, g2, g3), z)**2))

Eq(pw(2*z, g2, g3), (-g2/2 + 6*pw(z, g2, g3)**2)**2/(4*pwp(z, g2, g3)**2) - 2*pw(z, g2, g3))

Eq(pw(2*z, g2, g3), (-g2/2 + 6*pw(z, g2, g3)**2)**2/(4*(-g2*pw(z, g2, g3) - g3 + 4*pw(z, g2, g3)**3)) - 2*pw(z, g2, g3))

...

Eq(pw(z, g2, g3), Sum(z**(2*n)*(2*n + 1)*G[2*n + 2], (n, 1, oo)) + z**(-2))

Eq(1/sigma(z, g2, g3), 1/z + 6*g3*z**5/7 + g2*z**3/10 + O(z**7))

Eq(sigma(z, g2, g3), 85860*g2**2*g3**2*z**21/7 + 171*g2**2*g3*z**15/10 - g2**2*z**9/4 + 9936*g2*g3**2*z**17/17 - 18*g2*g3*z**11/11 - g2*z**5/10 - 216*g3**2*z**13/13 - 6*g3*z**7/7 + z)

Eq(sigma(2*m*omega3 + 2*n*omega1 + z, g2, g3), (-1)**(m*n + m + n)*sigma(z, g2, g3)*exp(2*eta1*m*n*omega3 + 2*eta1*n**2*omega1 + 2*eta1*n*z + 2*eta3*m**2*omega3 + 2*eta3*m*n*omega1 + 2*eta3*m*z))

Eq(f(x + a), Sum(x**n*Subs(Derivative(f(x), (x, n)), x, a)/factorial(n), (n, 0, N)))

In [4]:
log_sigma_xy_a = Eq(Derivative(log(sigma(x+y,g2,g3)/sigma(y,g2,g3)),y), zw(x+y,g2,g3) - zw(y,g2,g3))
log_sigma_xy_b = Eq(Derivative(log(sigma(x+y,g2,g3)/sigma(y,g2,g3)*exp(-zw(x,g2,g3)*y)),y), 
   -zw(x, g2, g3) + zw(x+y,g2,g3) - zw(y,g2,g3))

pwp_x_y_log_sigma_diff = Eq(pwp(x, g2, g3)/((pw(x, g2, g3) - pw(y, g2, g3))),
 2*Derivative(log(sigma(x+y,g2,g3)/sigma(y,g2,g3)*exp(-zw(x,g2,g3)*y)),y) 
   - Derivative(log(pw(x,g2,g3) - pw(y,g2,g3)),y)
  )

pwp_x_y_log_sigma_diff_b = Eq(pwp_x_y_log_sigma_diff.lhs,
 2*Derivative(log(sigma(x + y, g2, g3)*exp(-y*zw(x, g2, g3))/sigma(y, g2, g3)/sigma(x, g2, g3)), y) - 
   Derivative(log(sigma(x + y, g2, g3)*exp(-y*zw(x, g2, g3))/
                  (sigma(x, g2, g3)*sigma(y, g2, g3))), y)-
   Derivative(log(sigma(-x + y, g2, g3)*exp(y*zw(x, g2, g3))/
                  (sigma(x, g2, g3)*sigma(y, g2, g3))), y)
  
)
pwp_x_y_log_sigma_diff_c = Eq(pwp(x, g2, g3)/(pw(x, g2, g3) - pw(y, g2, g3)),
    Derivative(log(sigma(x + y, g2, g3)*exp(-2*y*zw(x, g2, g3))/sigma(-x + y, g2, g3)), y)
)

pw_to_zw_identity
log_sigma_xy_a
log_sigma_xy_b
pwp_x_y_log_sigma_diff
pwp_x_y_log_sigma_diff_b
pwp_x_y_log_sigma_diff_c

Eq((pwp(x, g2, g3) - pwp(y, g2, g3))/(2*(pw(x, g2, g3) - pw(y, g2, g3))), -zw(x, g2, g3) - zw(y, g2, g3) + zw(x + y, g2, g3))

Eq(Derivative(log(sigma(x + y, g2, g3)/sigma(y, g2, g3)), y), -zw(y, g2, g3) + zw(x + y, g2, g3))

Eq(Derivative(log(sigma(x + y, g2, g3)*exp(-y*zw(x, g2, g3))/sigma(y, g2, g3)), y), -zw(x, g2, g3) - zw(y, g2, g3) + zw(x + y, g2, g3))

Eq(pwp(x, g2, g3)/(pw(x, g2, g3) - pw(y, g2, g3)), 2*Derivative(log(sigma(x + y, g2, g3)*exp(-y*zw(x, g2, g3))/sigma(y, g2, g3)), y) - Derivative(log(pw(x, g2, g3) - pw(y, g2, g3)), y))

Eq(pwp(x, g2, g3)/(pw(x, g2, g3) - pw(y, g2, g3)), -Derivative(log(sigma(-x + y, g2, g3)*exp(y*zw(x, g2, g3))/(sigma(x, g2, g3)*sigma(y, g2, g3))), y) + Derivative(log(sigma(x + y, g2, g3)*exp(-y*zw(x, g2, g3))/(sigma(x, g2, g3)*sigma(y, g2, g3))), y))

Eq(pwp(x, g2, g3)/(pw(x, g2, g3) - pw(y, g2, g3)), Derivative(log(sigma(x + y, g2, g3)*exp(-2*y*zw(x, g2, g3))/sigma(-x + y, g2, g3)), y))

## Frobenius Stickelberger determinant formula for Kronecker theta functions

In [5]:
sigma_product_f_s = (
    (-1)**(N*(N-1)/2)*
    Product(factorial(k),(k,1,N))*
    sigma(Sum(mu[j],(j,0,N)),g2,g3)*
    Product(Product(Piecewise((sigma(mu[l] - mu[m], g2, g3),l<m),(1,True)),(l,0,N)),(m,0,N))/
    Product(sigma(mu[n],g2,g3)**(N+1),(n,0,N))
)

def frob_stick(Nval, evaluated=False, WPversion=False):
    """
    Frobenius Stickelberger in its original form
    """
    if evaluated:
        return Eq(p_matrix_f_s(Nval).det(), sigma_product_f_s.subs(N,Nval).doit())
    elif WPversion:
        return Eq(WPdet(Nval), sigma_product_f_s.subs(N,Nval).doit())
    return Eq(Det(p_matrix_f_s(Nval)), sigma_product_f_s.subs(N,Nval).doit())

In [6]:
sigma_product_f_s_N_factored = (
    (-1)**(N*(N-1)/2)*
    Product(factorial(k),(k,1,N-1))*
    factorial(N)*
    sigma(Sum(mu[j],(j,0,N)),g2,g3)*
    Product(Product(Piecewise((sigma(mu[l] - mu[m], g2, g3),l<m),(1,True)),(l,0,N-1)),(m,0,N-1))*
    Product(Piecewise((sigma(mu[l] - mu[N], g2, g3),l<N),(1,True)),(l,0,N))/
    Product(sigma(mu[n],g2,g3)**(N),(n,0,N))/
    Product(sigma(mu[n],g2,g3),(n,0,N))
)

sigma_product_f_s_N_over_N_minus_1 = (sigma_product_f_s_N_factored/sigma_product_f_s.subs(N,N-1))

subs1_list = [
    (Product(Piecewise((sigma(mu[l] - mu[N], g2, g3),l<N),(1,True)),(l,0,N)), 
     (-1)**N*Product(sigma(z - xi[l], g2, g3),(l,0,N))/sigma(z - xi[N], g2, g3)),
    (Product(sigma(mu[n],g2,g3),(n,0,N)),Product(sigma(xi[n],g2,g3),(n,0,N))*sigma(z,g2,g3)/sigma(xi[N],g2,g3)),
    (sigma(Sum(mu[j],(j,0,N-1)),g2,g3),-sigma(xi[N],g2,g3)),
    (Sum(mu[j],(j,0,N)),-xi[N]+z),
    (sigma(mu[N],g2,g3),sigma(z,g2,g3))
]
subs2_list = [
    ((-1)**(2*N)*Product(sigma(z - xi[l], g2, g3),(l,0,N)), 
   Product(sigma(z - xi[l], g2, g3)/sigma(xi[l], g2, g3)/sigma(z, g2, g3),(l,0,N)) *
   Product(sigma(xi[n], g2, g3),(n,0,N)) *
   (Product(sigma(z, g2, g3),(l,0,N)).doit()))
]

wp_sigma_product_fs_N_over_N_minus_1_mu_n = Eq(WPdet(N)/WPdet(N-1), sigma_product_f_s_N_over_N_minus_1).subs(
    Product(sigma(mu[n],g2,g3)**N,(n,0,N)),sigma(mu[N],g2,g3)**N*Product(sigma(mu[n],g2,g3)**N,(n,0,N-1))
).expand()

wp_sigma_product_fs_N_over_N_minus_1 = (
    wp_sigma_product_fs_N_over_N_minus_1_mu_n
    .subs(subs1_list)
    .subs(subs2_list)
).expand()
sigma_product_f_s_N_over_N_minus_1 = wp_sigma_product_fs_N_over_N_minus_1.rhs


def p_matrix_f_s(N): 
    return Matrix([[1, *[pw(x,g2,g3).diff((x,j)).subs(x,mu[k]) for j in range(N)]] for k in range(N+1)])

def p_matrix_f_s_1dz(N): 
    return Matrix([*[[1, *[pw(x,g2,g3).diff((x,j)).subs(x,mu[k]) for j in range(N)]] for k in range(N)],
       [1, *[pw(z,g2,g3).diff((z,j)) for j in range(N)]]])

def frob_stick_1dz_ratio(Nval, evaluated=False):
    """
    Frobenius Stickelberger adapted for Kronecker theta functions
    """
    if evaluated:
        return Eq(
            (((-1)**(Nval-1)/factorial(Nval) * p_matrix_f_s_1dz(Nval).det()/p_matrix_f_s(Nval-1).det()).expand())
                  .collect([pw(z,g2,g3), diff(pw(z,g2,g3),z)],factor), 
            (-1)**(Nval-1)/factorial(Nval) * sigma_product_f_s_N_over_N_minus_1.subs(N,Nval).doit().subs(
            xi[Nval],-Sum(xi[j],(j,0,Nval-1)).doit()
        ).subs([(xi[j],mu[j]) for j in range(Nval)]))
    
    return Eq(
        (-1)**(Nval-1)/factorial(Nval) * Det(p_matrix_f_s_1dz(Nval))/Det(p_matrix_f_s(Nval-1)), 
        
        (-1)**(Nval-1)/factorial(Nval) * sigma_product_f_s_N_over_N_minus_1.subs(N,Nval).doit().subs(
            xi[Nval],-Sum(xi[j],(j,0,Nval-1)).doit()
        ).subs([(xi[j],mu[j]) for j in range(Nval)]))

## General completely degenerate case

In [7]:
def diff_mu_0_deg(Nval):
    return Matrix([
        [1 if j == 0 else 0, *[diff(pw(mu[0], g2, g3), (mu[0], i + j)) for i in range(Nval - 1)]]
        for j in range(Nval)
    ])

def diff_mu_0_z_deg(Nval):
    return Matrix([
        *[
            [1 if j == 0 else 0, *[diff(pw(mu[0], g2, g3), (mu[0], i + j)) for i in range(Nval)]]
            for j in range(Nval)
        ],
        [1, *[diff(pw(z, g2, g3), (z, i)) for i in range(Nval)]]
   ])

def det_ratio_deg_case(Nval, evaluated=False):
    if evaluated:
        return Eq(1/factorial(Nval) * det(diff_mu_0_z_deg(Nval))/det(diff_mu_0_deg(Nval)), 
                  -sigma(z - mu[0], g2, g3)**Nval*sigma(z + Nval*mu[0], g2, g3)
                  /sigma(Nval*mu[0], g2, g3)/sigma(mu[0], g2, g3)**Nval/sigma(z, g2, g3)**(Nval +1))
    return Eq(1/factorial(Nval) * Det(diff_mu_0_z_deg(Nval))/Det(diff_mu_0_deg(Nval)),
              -sigma(z - mu[0], g2, g3)**Nval*sigma(z + Nval*mu[0], g2, g3)
                  /sigma(Nval*mu[0], g2, g3)/sigma(mu[0], g2, g3)**Nval/sigma(z, g2, g3)**(Nval+1))
    

In [8]:
sigma_product_f_sb = (
    (-1)**(N*(N-1)/2)*
    Product(factorial(k),(k,1,N))*
    sigma(Sum(mu[0] + epsilon[j],(j,0,N)),g2,g3)*
    Product(Product(Piecewise((sigma(mu[0] + epsilon[l] - mu[0] - epsilon[m], g2, g3),l<m),(1,True)),(l,0,N)),(m,0,N))/
    Product(sigma(mu[0] + epsilon[n],g2,g3)**(N+1),(n,0,N))
)
sigma_product_f_sc = (
    (-1)**(N*(N-1)/2)*
    Product(factorial(k),(k,1,N))*
    sigma(Sum(mu[0],(j,0,N)).doit(),g2,g3)*
    Product(Product(Piecewise((sigma(mu[0] + epsilon[l] - mu[0] - epsilon[m], g2, g3),l<m),(1,True)),(l,0,N)),(m,0,N))/
    Product(sigma(mu[0],g2,g3)**(N+1),(n,0,N)).doit()
)

sigma_product_f_sd = (
    (-1)**(N*(N-1)/2)*
    Product(factorial(k),(k,1,N))**2 *
    sigma((N+1)*mu[0],g2,g3)*
    Product(Product(Piecewise((sigma(epsilon[l] - epsilon[m], g2, g3),l<m),(1,True)),(l,0,N)),(m,0,N))/
    sigma(mu[0],g2,g3)**(N+1)**2/Product(epsilon[j]**j, (j,1,N))
)

sigma_product_f_se = (
    (-1)**(N**2)*Product(factorial(k),(k,1,N))**2 * sigma((N+1)*mu[0],g2,g3)/sigma(mu[0],g2,g3)**(N+1)**2
)

In [9]:
def det_const_deg_case(Nval, evaluated=False):
    if evaluated:
        return Eq(det(diff_mu_0_deg(Nval)), 
            (-1)**((Nval-1)**2)*product(factorial(k),(k,1,Nval - 1))**2 * 
            sigma(Nval*mu[0],g2,g3)/sigma(mu[0],g2,g3)**((Nval)**2)
                 )
    return Eq(Det(diff_mu_0_deg(Nval)),
        (-1)**((Nval-1)**2)*product(factorial(k),(k,1,Nval - 1))**2 * 
            sigma(Nval*mu[0],g2,g3)/sigma(mu[0],g2,g3)**((Nval)**2)
             )

In [10]:
def det_ratio_deg_case(Nval, evaluated=False):
    if evaluated:
        return Eq(1/factorial(Nval) * det(diff_mu_0_z_deg(Nval))/det(diff_mu_0_deg(Nval)), 
                  -sigma(z - mu[0], g2, g3)**Nval*sigma(z + Nval*mu[0], g2, g3)
                  /sigma(Nval*mu[0], g2, g3)/sigma(mu[0], g2, g3)**Nval/sigma(z, g2, g3)**(Nval +1))
    return Eq(1/factorial(Nval) * Det(diff_mu_0_z_deg(Nval))/Det(diff_mu_0_deg(Nval)),
              -sigma(z - mu[0], g2, g3)**Nval*sigma(z + Nval*mu[0], g2, g3)
                  /sigma(Nval*mu[0], g2, g3)/sigma(mu[0], g2, g3)**Nval/sigma(z, g2, g3)**(Nval+1))

## Numerical test

In [11]:
def mpc_to_float(mpc_val):
    return float(mpc_val.real) + float(mpc_val.imag)*1j

def sigma_prod_const(_z, _N, omegas):
    return (    
        (-1)**((_N - 1)**2)*product(factorial(k),(k,1,_N - 1))**2 * 
        we.wsigma(_N *_z, omegas)/we.wsigma(_z, omegas)**(_N**2)
    ).expand().evalf()

def test_const_det_sigma_power(_z, _g2, _g3, _N):
    omega_ = we.omega_from_g(_g2, _g3)
    omega1_ = mpc_to_float(omega_[0])
    omega2_ = mpc_to_float(omega_[1])
    omega3_ = mpc_to_float(omega_[2])
    if im(omega2_/omega1_) <= 0:
        omega2_ = -omega2_
    omegas_ = (omega1_, omega2_)  

    pw_val = mpc_to_float(we.wp(_z, omegas_))
    pwp_val = mpc_to_float(we.wpprime(_z, omegas_))

    sigma_N_val = sigma_prod_const(_z, _N, omegas_)

    det_N_val_ = (
        det_const_deg_case(Nval=_N, evaluated=False).lhs.args[0]
        .subs([wpk(_i).subs(z,mu[0]).args for _i in range(1, 2*_N - 1)][::-1])
        .subs([(pw(mu[0],g2,g3),pw_val), (pwp(mu[0],g2,g3),pwp_val), (g2, _g2), (g3, _g3)])
        .expand()
        .det()
    )

    err_val = abs(sigma_N_val - det_N_val_)/(sigma_N_val + det_N_val_)
    
    return {
        "sigma_N_val": sigma_N_val,
        "det_N_val_": det_N_val_,
        "err_val": err_val
    }


def sigma_prod(_z, _mu0, _N, omegas): 
    return (    
        -we.wsigma(_z - _mu0, omegas)**_N*we.wsigma(_z + _N*_mu0, omegas)/
        we.wsigma(_N*_mu0, omegas)/
        we.wsigma(_mu0, omegas)**_N/
        we.wsigma(_z, omegas)**(_N+1)
    )

def test_det_ratio(_z, _mu0, _g2, _g3, _N):
    omega_ = we.omega_from_g(_g2, _g3)
    omega1_ = mpc_to_float(omega_[0])
    omega2_ = mpc_to_float(omega_[1])
    omega3_ = mpc_to_float(omega_[2])
    if im(omega2_/omega1_) <= 0:
        omega2_ = -omega2_
    omegas_ = (omega1_, omega2_)  

    pw_val_mu0 = mpc_to_float(we.wp(_mu0, omegas_))
    pwp_val_mu0 = mpc_to_float(we.wpprime(_mu0, omegas_))
    pw_val_z = mpc_to_float(we.wp(_z, omegas_))
    pwp_val_z = mpc_to_float(we.wpprime(_z, omegas_))

    sigma_N_val = sigma_prod(_z, _mu0, _N, omegas_)

    det_ratio_N_val_ = ( 
        diff_mu_0_z_deg(_N)
        .subs([wpk(_i).subs(z,mu[0]).args for _i in range(1, 2*_N - 1)][::-1])
        .subs([wpk(_i).args for _i in range(1, 2*_N - 1)][::-1])
        .subs([(pw(mu[0],g2,g3), pw_val_mu0), (pwp(mu[0],g2,g3),pwp_val_mu0)])
        .subs([(pw(z,g2,g3), pw_val_z), (pwp(z,g2,g3), pwp_val_z), (g2, _g2), (g3, _g3)])
        .expand()
        .det() /
        diff_mu_0_deg(_N)
        .subs([wpk(_i).subs(z,mu[0]).args for _i in range(1, 2*_N - 1)][::-1])
        .subs([wpk(_i).args for _i in range(1, 2*_N - 1)][::-1])
        .subs([(pw(mu[0],g2,g3), pw_val_mu0), (pwp(mu[0],g2,g3),pwp_val_mu0)])
        .subs([(pw(z,g2,g3), pw_val_z), (pwp(z,g2,g3), pwp_val_z), (g2, _g2), (g3, _g3)])
        .expand()
        .det() /
        factorial(_N)
   ).expand().evalf()

    err_val = abs(sigma_N_val - det_ratio_N_val_)/(sigma_N_val + det_ratio_N_val_)
    
    return {
        "sigma_N_val": sigma_N_val,
        "det_ratio_N_val_": det_ratio_N_val_,
        "err_val": err_val
    }

def num_evaluate_weierstrass(expression, values, show_expression=True):

    lhs_value = (expression.lhs
        .subs(values)
        .replace(sigma, we.wsigma_from_g2_g3)
        .replace(zw, we.wzeta_from_g2_g3)
        .replace(pw, we.wp_from_g2_g3)
        .replace(pwp, we.wpprime_from_g2_g3)
    )

    rhs_value = (expression.rhs
        .subs(values)
        .replace(sigma, we.wsigma_from_g2_g3)
        .replace(zw, we.wzeta_from_g2_g3)
        .replace(pw, we.wp_from_g2_g3)
        .replace(pwp, we.wpprime_from_g2_g3)
    )
    
    try:
        lhs_value = lhs_value.evalf()
    except:
        pass
    try:
        rhs_value = rhs_value.evalf()
    except:
        pass
        
    print('LHS', lhs_value)
    print('RHS', rhs_value)
    # Symmetric Mean Absolute Percentage Error (SMAPE)
    print('ERROR (SMAPE)', abs(lhs_value - rhs_value) * 2/(abs(lhs_value) + abs(rhs_value)))
    
    if show_expression:
        return expression

In [12]:
from random import random
g2val_ = (random() - 0.5) + (random() - 0.5)*1j
g3val_ = (random() - 0.5) + (random() - 0.5)*1j
zval_ = (random() - 0.5) + (random() - 0.5)*1j

omega_ = we.omega_from_g(g2val_, g3val_)
omega1_ = mpc_to_float(omega_[0])
omega2_ = mpc_to_float(omega_[1])
omega3_ = mpc_to_float(omega_[2])
if im(omega2_/omega1_) <= 0:
    omega2_ = -omega2_
omegas = (omega1_, omega2_)  

num_subs_simple = [
    (eta1, zw(omega1, g2, g3)), (g2, g2val_), (g3, g3val_), (omega1, omega1_), (z,zval_)
]

## The fith order case

The starting point is the Frobenius-Stickelberger determinant formula involving a product of five Kronecker theta functions in the limit that all the free $\mu_j$ parameters in the determinant tend to the same value (degeneracy).

In [13]:
N_order = 5

In [14]:
vals = [mu[_k] for _k in range(N_order)]
vals.append(z)
wpk_subs = [wpk(_n).subs(z, val) for val in vals for _n in range(1,N_order + 2)][::-1]

In [15]:
drdc_3 = det_ratio_deg_case(N_order - 1)
drdc_3_with_subs = drdc_3.subs([_eq.args for _eq in wpk_subs])

__xx = (drdc_3
        .subs(*det_const_deg_case(Nval=N_order - 1, evaluated=False).args)
        .subs([_eq.args for _eq in wpk_subs])
       )

drdc_3
drdc_3_with_subs
__xx

# det_ratio_deg_case(3).subs(*det_const_deg_case(Nval=3, evaluated=False).args)

Eq(Det(Matrix([
[1,                         pw(mu[0], g2, g3),      Derivative(pw(mu[0], g2, g3), mu[0]), Derivative(pw(mu[0], g2, g3), (mu[0], 2)), Derivative(pw(mu[0], g2, g3), (mu[0], 3))],
[0,      Derivative(pw(mu[0], g2, g3), mu[0]), Derivative(pw(mu[0], g2, g3), (mu[0], 2)), Derivative(pw(mu[0], g2, g3), (mu[0], 3)), Derivative(pw(mu[0], g2, g3), (mu[0], 4))],
[0, Derivative(pw(mu[0], g2, g3), (mu[0], 2)), Derivative(pw(mu[0], g2, g3), (mu[0], 3)), Derivative(pw(mu[0], g2, g3), (mu[0], 4)), Derivative(pw(mu[0], g2, g3), (mu[0], 5))],
[0, Derivative(pw(mu[0], g2, g3), (mu[0], 3)), Derivative(pw(mu[0], g2, g3), (mu[0], 4)), Derivative(pw(mu[0], g2, g3), (mu[0], 5)), Derivative(pw(mu[0], g2, g3), (mu[0], 6))],
[1,                             pw(z, g2, g3),              Derivative(pw(z, g2, g3), z),         Derivative(pw(z, g2, g3), (z, 2)),         Derivative(pw(z, g2, g3), (z, 3))]]))*Det(Matrix([
[1,                         pw(mu[0], g2, g3),      Derivative(pw(mu[0], g2, g3), mu

Eq(Det(Matrix([
[1,                       pw(mu[0], g2, g3),                                          pwp(mu[0], g2, g3),                              -g2/2 + 6*pw(mu[0], g2, g3)**2,                                                       12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3)],
[0,                      pwp(mu[0], g2, g3),                              -g2/2 + 6*pw(mu[0], g2, g3)**2,                     12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3),                                   -18*g2*pw(mu[0], g2, g3) - 12*g3 + 120*pw(mu[0], g2, g3)**3],
[0,          -g2/2 + 6*pw(mu[0], g2, g3)**2,                     12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3), -18*g2*pw(mu[0], g2, g3) - 12*g3 + 120*pw(mu[0], g2, g3)**3,                                         -18*(g2 - 20*pw(mu[0], g2, g3)**2)*pwp(mu[0], g2, g3)],
[0, 12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3), -18*g2*pw(mu[0], g2, g3) - 12*g3 + 120*pw(mu[0], g2, g3)**3,       -18*(g2 - 20*pw(mu[0], g2, g3)**2)*pwp(mu[0], g2, g3), 9*g2**2 - 1008*g2*pw(mu[0], 

Eq(-sigma(mu[0], g2, g3)**16*Det(Matrix([
[1,                       pw(mu[0], g2, g3),                                          pwp(mu[0], g2, g3),                              -g2/2 + 6*pw(mu[0], g2, g3)**2,                                                       12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3)],
[0,                      pwp(mu[0], g2, g3),                              -g2/2 + 6*pw(mu[0], g2, g3)**2,                     12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3),                                   -18*g2*pw(mu[0], g2, g3) - 12*g3 + 120*pw(mu[0], g2, g3)**3],
[0,          -g2/2 + 6*pw(mu[0], g2, g3)**2,                     12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3), -18*g2*pw(mu[0], g2, g3) - 12*g3 + 120*pw(mu[0], g2, g3)**3,                                         -18*(g2 - 20*pw(mu[0], g2, g3)**2)*pwp(mu[0], g2, g3)],
[0, 12*pw(mu[0], g2, g3)*pwp(mu[0], g2, g3), -18*g2*pw(mu[0], g2, g3) - 12*g3 + 120*pw(mu[0], g2, g3)**3,       -18*(g2 - 20*pw(mu[0], g2, g3)**2)*pwp(mu[0], g2, g3), 9*

The final level of degeneracy is achieved by making the parameter $\mu_0$ take the appropriate rational multiple of a period $\omega_i$ such that the right hand side of the determinat formula is a monomial in a Kronecker theta function. If $k$ is the order of the monomial, the following condition must be satisifed:

In [16]:
Eq((k-1)*mu[0],2*omega1 - mu[0])
Eq(mu[0],2*omega1/k)

full_deg_mu0_5 = Eq(mu[0],2*omega1/N_order)

_qq_a = quasi_period.subs([(m,0),(n,1),(z,z-2*omega1/N_order)])
_qq_b = Eq(_qq_a.lhs/_qq_a.lhs.subs(z,0), (_qq_a.rhs/_qq_a.rhs.subs(z,0)).simplify()).subs([
    (sigma(-2*omega1/N_order, g2, g3), -sigma(2*omega1/N_order, g2, g3))
])
_qq_b_min = _qq_b.subs(z,-z)

Uz_sig = Eq(U(z), sigma(-2*omega1/5 + z, g2, g3)*exp(2*eta1*z/5)/(sigma(2*omega1/5, g2, g3)*sigma(z, g2, g3)))
Vz_sig = Eq(V(z), sigma(2*omega1/5 + z, g2, g3)*exp(-2*eta1*z/5)/(sigma(2*omega1/5, g2, g3)*sigma(z, g2, g3)))
UVz_sig = Eq(U(z)*V(z),(U(z)*V(z)).subs([Uz_sig.args, Vz_sig.args]))
UVz_pw = UVz_sig.subs(*sigma_p_identity.subs([(y,2*omega1/5),(x,z)]).args[::-1])

full_deg_mu0_5
_qq_b
_qq_b_min
Uz_sig
Vz_sig
Eq(U(z), V(-z))
UVz_sig
UVz_pw

Eq((k - 1)*mu[0], 2*omega1 - mu[0])

Eq(mu[0], 2*omega1/k)

Eq(mu[0], 2*omega1/5)

Eq(sigma(8*omega1/5 + z, g2, g3)/sigma(8*omega1/5, g2, g3), -sigma(-2*omega1/5 + z, g2, g3)*exp(2*eta1*z)/sigma(2*omega1/5, g2, g3))

Eq(sigma(8*omega1/5 - z, g2, g3)/sigma(8*omega1/5, g2, g3), -sigma(-2*omega1/5 - z, g2, g3)*exp(-2*eta1*z)/sigma(2*omega1/5, g2, g3))

Eq(U(z), sigma(-2*omega1/5 + z, g2, g3)*exp(2*eta1*z/5)/(sigma(2*omega1/5, g2, g3)*sigma(z, g2, g3)))

Eq(V(z), sigma(2*omega1/5 + z, g2, g3)*exp(-2*eta1*z/5)/(sigma(2*omega1/5, g2, g3)*sigma(z, g2, g3)))

Eq(U(z), V(-z))

Eq(U(z)*V(z), sigma(-2*omega1/5 + z, g2, g3)*sigma(2*omega1/5 + z, g2, g3)/(sigma(2*omega1/5, g2, g3)**2*sigma(z, g2, g3)**2))

Eq(U(z)*V(z), pw(2*omega1/5, g2, g3) - pw(z, g2, g3))

By Laplace expansion along the bottom row of the detreminant, it is clear that this determinant formula takes the form shown below. There are a few different ways to find expressions for the coefficients $c_j$. They could be expressed in terms of $\mathrm{pw}(2\omega_1/5,g_2,g_3), \mathrm{pwp}(2\omega_1/5,g_2,g_3), g_2, g_3$ via the co-factors in the determinant, but that often gives quite complicated expressions. With some patience, it is often possible to find simpler expressions by other means as shown below.

In [17]:
c_unknown_sigma = Eq(c[0] + c[1]*pw(z,g2,g3) + c[2]*pwp(z,g2,g3) + c[3]*pw(z,g2,g3)**2 + pw(z,g2,g3)*pwp(z,g2,g3)/2,
  __xx.rhs.subs(mu[0],2*omega1/5).subs(*_qq_b.args))
c_unknown_U = c_unknown_sigma.subs(Uz_sig.rhs, Uz_sig.lhs)
c_unknown_V = (c_unknown_U.subs(z,-z)
               .subs([(pw(-z,g2,g3), pw(z,g2,g3)), (pwp(-z,g2,g3), -pwp(z,g2,g3)), (U(-z),V(z))]))

pw_pwp_sig5_even = Eq(c_unknown_sigma.lhs + c_unknown_sigma.lhs.subs(z,-z), 
   c_unknown_sigma.rhs + c_unknown_sigma.rhs.subs(z,-z)).subs(
    [(pw(-z,g2,g3),pw(z,g2,g3)), (pwp(-z,g2,g3),-pwp(z,g2,g3)), (sigma(-z,g2,g3),-sigma(z,g2,g3))])


c_unknown_sigma
c_unknown_U
c_unknown_V
pw_pwp_sig5_even

Eq(pw(z, g2, g3)**2*c[3] + pw(z, g2, g3)*pwp(z, g2, g3)/2 + pw(z, g2, g3)*c[1] + pwp(z, g2, g3)*c[2] + c[0], sigma(-2*omega1/5 + z, g2, g3)**5*exp(2*eta1*z)/(sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)**5))

Eq(pw(z, g2, g3)**2*c[3] + pw(z, g2, g3)*pwp(z, g2, g3)/2 + pw(z, g2, g3)*c[1] + pwp(z, g2, g3)*c[2] + c[0], U(z)**5)

Eq(pw(z, g2, g3)**2*c[3] - pw(z, g2, g3)*pwp(z, g2, g3)/2 + pw(z, g2, g3)*c[1] - pwp(z, g2, g3)*c[2] + c[0], V(z)**5)

Eq(2*pw(z, g2, g3)**2*c[3] + 2*pw(z, g2, g3)*c[1] + 2*c[0], -sigma(-2*omega1/5 - z, g2, g3)**5*exp(-2*eta1*z)/(sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)**5) + sigma(-2*omega1/5 + z, g2, g3)**5*exp(2*eta1*z)/(sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)**5))

In [18]:
# g2_g3_c_eq = Eq((c_unknown_U.lhs*c_unknown_V.lhs - c_unknown_U.rhs*c_unknown_V.rhs)
#    .subs(*UVz_pw.args)
#    .expand()
#    .subs(*pwp_sqrd.args)
#    .expand()
#    .collect(pw(z,g2,g3), factor)
#    ,0)

# c_coeff_eqs = [g2_g3_c_eq.lhs.coeff(pw(z,g2,g3),i) for i in range(5)]
# solve(c_coeff_eqs,[c[0], c[1],c[2],c[3],c[4]])

## Finding the $c_j$ coefficients to weight $\mathrm{pw}(z,g_2,g_3), \mathrm{pwp}(z,g_2,g_3)$

### Coefficient $c_3$

In [19]:
c3_limit_a = Eq(Limit(pw(z,g2,g3)**2*sigma(z,g2,g3)**4*c[3], z,0),
   Limit((sigma(z,g2,g3)**4*pw_pwp_sig5_even.rhs/2).expand(),z,0))

c3_limit_b = Eq(Limit(pw(z,g2,g3)**2*sigma(z,g2,g3)**4*c[3], z,0),
   Limit((sigma(z,g2,g3)**5*pw_pwp_sig5_even.rhs/z/2).expand(),z,0))

G4_g2_60 = Eq(G[4], g2/60)
lim_sigma_0 = Eq(Limit(sigma(z,g2,g3),z,0), z)
pw_diff_2_at_zero = pw_diff_at_zero.subs(j,2).subs(*G4_g2_60.args)

pw_sqrd = Eq((diff(pw(z,g2,g3),z,2).subs([_eq.args for _eq in wpk_subs])  + g2/2)/6, 
             (diff(pw(z,g2,g3),z,2) + g2/2)/6)
pw_sqrd_nr_0 = Eq(Limit(pw_sqrd.lhs,z,0), 
                  g2/12 + Limit(6*(pw_sqrd.rhs -g2/12),z,0)/6).subs(*pw_diff_2_at_zero.args)

f_diff_def = Eq(Limit((f(z) - f(-z))/z/2,z,0), limit((f(z) - f(-z))/2/z,z,0))

c3_as_diff = Eq(c[3], 
                Derivative((sigma(-2*omega1/5 + z, g2, g3)*exp(2*eta1*z/5)/sigma(2*omega1/5, g2, g3))**5,z)
                .subs(z,0)
               )

c3_as_ln_diff_a = Eq(c[3], 
                (Derivative(ln((sigma(-2*omega1/5 + z, g2, g3)*exp(2*eta1*z/5)/sigma(2*omega1/5, g2, g3))**5),z)
                 *(sigma(-2*omega1/5 + z, g2, g3)*exp(2*eta1*z/5)/sigma(2*omega1/5, g2, g3))**5)
                .subs(z,0)
               )
c3_as_ln_diff_b = Eq(c[3], 
                -(5*Derivative(ln(sigma(-2*omega1/5 + z, g2, g3)),z) + diff(2*eta1*z,z)).subs(z,0)
               )
c3_zeta = Eq(c[3], -2*eta1 + 5*zw(2*omega1/5, g2, g3))

# c3_limit_a
# lim_sigma_0
# c3_limit_b
# pw_sqrd
# pw_diff_2_at_zero
# pw_sqrd_nr_0
# f_diff_def
# c3_as_diff
# c3_as_ln_diff_a
# c3_as_ln_diff_b
c3_zeta

Eq(c[3], -2*eta1 + 5*zw(2*omega1/5, g2, g3))

### Coefficient $c_2$

In [20]:
pwp_c_unknown = Eq(pwp(z,g2,g3)*c + diff(pw(z,g2,g3),(z,3))/24 ,(__xx.rhs - __xx.rhs.subs(z,-z))/2)
pwp_c_unknown_eval = pwp_c_unknown.subs([_eq.args for _eq in wpk_subs])
c_sol = Eq(c, solve(pwp_c_unknown_eval.subs(z,mu[0]).subs(sigma(0,g2,g3),0),c)[0])
pwp_c_known = pwp_c_unknown_eval.subs(*c_sol.args)



pwp_c_known_deg = pwp_c_known.subs(*full_deg_mu0_5.args)

_pwp_omega1 = Eq(pwp(2*omega1/N_order,g2,g3), - sigma(4*omega1/N_order,g2,g3)/sigma(2*omega1/N_order,g2,g3)**4)
pwp_c_known_deg_b = pwp_c_known_deg.subs([
    _qq_b.args, 
    _qq_b_min.args,
    _qq_a.subs(z,0).args,
    (sigma(-2*omega1/N_order, g2, g3), -sigma(2*omega1/N_order, g2, g3)),
    _qq_a.subs(z,-2*omega1/5).args,
    _pwp_omega1.args,
    (sigma(-4*omega1/N_order, g2, g3), -sigma(4*omega1/N_order, g2, g3)),
    
])

UV_4_5_omega = (UVz_sig
 .subs(z,omega1*4/5)
 .subs(*_qq_a.expand().subs(z,-2*omega1/5).args)
 .subs(sigma(-4*omega1/5,g2,g3), -sigma(4*omega1/5,g2,g3))
 .subs(sigma(4*omega1/5, g2, g3), solve(_pwp_omega1, sigma(4*omega1/5, g2, g3))[0])
)


sigma_c_UV_pwp = Eq(
    sigma(4*omega1/5, g2, g3)**4*exp(-4*eta1*omega1/5)/(2*sigma(2*omega1/5, g2, g3)**6),
    (sigma(4*omega1/5, g2, g3)**4*exp(-4*eta1*omega1/5)/(2*sigma(2*omega1/5, g2, g3)**6))
    .subs(sigma(4*omega1/5, g2, g3), solve(_pwp_omega1, sigma(4*omega1/5, g2, g3))[0])
    *(UV_4_5_omega.rhs**2/UV_4_5_omega.lhs**2)
)


sigma_c_pw_pwp = sigma_c_UV_pwp.subs(*UVz_pw.subs(z, 4*omega1/5).args)


pwp_z_UV_sig = pwp_c_known_deg_b.subs(*sigma_c_pw_pwp.args).subs([
    (sigma(-2*omega1/5 - z, g2, g3), -sigma(2*omega1/5 + z, g2, g3)),
    (sigma(- z, g2, g3), -sigma(z, g2, g3))
])
 

c2_pw_pwp = Eq(c[2], pwp_z_UV_sig.lhs.subs(pw(z,g2,g3),0)/pwp(z,g2,g3))

# Eq((c_unknown_sigma.lhs - c_unknown_sigma.lhs.subs(z,-z))/2, 
#    (c_unknown_sigma.rhs - c_unknown_sigma.rhs.subs(z,-z))/2).subs([
#     (pwp(-z,g2,g3), -pwp(z,g2,g3)), (pw(-z,g2,g3), pw(z,g2,g3))
# ])
# pwp_z_UV_sig


# pwp_c_unknown
# pwp_c_unknown_eval
# c_sol
# pwp_c_known
# pwp_c_known_deg
# _pwp_omega1
# pwp_c_known_deg_b
# UVz_sig.subs(z,omega1*4/5)
# UV_4_5_omega
# sigma_c_UV_pwp
# sigma_c_pw_pwp
# pwp_z_UV_sig
c2_pw_pwp

Eq(c[2], -pw(2*omega1/5, g2, g3)/2 + pwp(2*omega1/5, g2, g3)**2/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2))

In [21]:
UV_4_5_omega.subs(*UVz_pw.subs(z,4*omega1/5).args)

Eq(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3), -exp(2*eta1*omega1/5)/(pwp(2*omega1/5, g2, g3)*sigma(2*omega1/5, g2, g3)**5))

In [22]:
num_evaluate_weierstrass(pwp_c_known.subs(mu[0], 2*omega1/5), num_subs_simple)
num_evaluate_weierstrass(UVz_sig.subs(z,omega1*4/5).subs(*UVz_pw.subs(z,4*omega1/5).args), num_subs_simple)
num_evaluate_weierstrass(pwp_c_known_deg_b, num_subs_simple)
num_evaluate_weierstrass(sigma_c_pw_pwp, num_subs_simple)
num_evaluate_weierstrass(pwp_z_UV_sig, num_subs_simple)
num_evaluate_weierstrass(_pwp_omega1, num_subs_simple)

LHS 877.351685055411 + 1101.35880241375*I
RHS 877.351685055412 + 1101.35880241375*I
ERROR (SMAPE) 2.91104746121715e-16


Eq((-pw(2*omega1/5, g2, g3)/2 + sigma(-4*omega1/5, g2, g3)**4*sigma(6*omega1/5, g2, g3)/(2*pwp(2*omega1/5, g2, g3)*sigma(-2*omega1/5, g2, g3)**5*sigma(2*omega1/5, g2, g3)**4*sigma(8*omega1/5, g2, g3)))*pwp(z, g2, g3) + pw(z, g2, g3)*pwp(z, g2, g3)/2, -sigma(-2*omega1/5 + z, g2, g3)**4*sigma(8*omega1/5 + z, g2, g3)/(2*sigma(2*omega1/5, g2, g3)**4*sigma(8*omega1/5, g2, g3)*sigma(z, g2, g3)**5) + sigma(-2*omega1/5 - z, g2, g3)**4*sigma(8*omega1/5 - z, g2, g3)/(2*sigma(2*omega1/5, g2, g3)**4*sigma(8*omega1/5, g2, g3)*sigma(-z, g2, g3)**5))

LHS 1.07294016139886 + 0.772236847502549*I
RHS 1.07294016139886 + 0.772236847502548*I
ERROR (SMAPE) 3.35935092746323e-16


Eq(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3), sigma(6*omega1/5, g2, g3)/(sigma(2*omega1/5, g2, g3)*sigma(4*omega1/5, g2, g3)**2))

LHS 877.351685055411 + 1101.35880241375*I
RHS 877.351685055412 + 1101.35880241375*I
ERROR (SMAPE) 1.80535499614329e-16


...

Eq(sigma(4*omega1/5, g2, g3)**4*exp(-4*eta1*omega1/5)/(2*sigma(2*omega1/5, g2, g3)**6), pwp(2*omega1/5, g2, g3)**2/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2))

LHS 877.351685055412 + 1101.35880241375*I
RHS 877.351685055412 + 1101.35880241375*I
ERROR (SMAPE) 1.61475859795811e-16


Eq((-pw(2*omega1/5, g2, g3)/2 + pwp(2*omega1/5, g2, g3)**2/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2))*pwp(z, g2, g3) + pw(z, g2, g3)*pwp(z, g2, g3)/2, sigma(-2*omega1/5 + z, g2, g3)**5*exp(2*eta1*z)/(2*sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)**5) - sigma(2*omega1/5 + z, g2, g3)**5*exp(-2*eta1*z)/(2*sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)**5))

LHS (-2.94958702333146 - 4.0315850216109j)
RHS -2.94958702333145 - 4.0315850216109*I
ERROR (SMAPE) 1.77800255941285e-16


Eq(pwp(2*omega1/5, g2, g3), -sigma(4*omega1/5, g2, g3)/sigma(2*omega1/5, g2, g3)**4)

In [23]:
pwp_z_UV_sig_z_0_rhs_z = Eq((pwp_z_UV_sig.lhs/pwp(z,g2,g3)).simplify() - pw(z,g2,g3)/2,
   (pwp_z_UV_sig.rhs/pwp(z,g2,g3)).subs(*pwp_sigma_dbl_ratio.args).factor().expand().collect([
       sigma(-2*omega1/5 + z,g2,g3), sigma(2*omega1/5 + z,g2,g3)
   ], simplify)
                            - pw(z,g2,g3)/2
  )

sigma_a_taylor_omega_z0 = (
    taylor_series_b.subs(N,2).doit()
    .subs([(f(a),sigma(a, g2, g3)), (f(x),sigma(x, g2, g3)), (f(x+a),sigma(x+a, g2, g3))])
    .subs([(x,z),(a,-2*omega1/5)])
    .subs([d_sigma_zw_2.args, d_sigma_zw.args]).doit()
    .subs([(sigma(-2*omega1/5,g2,g3), -sigma(2*omega1/5,g2,g3)), (zw(-2*omega1/5,g2,g3), -zw(2*omega1/5,g2,g3))])
)
sigma_b_taylor_omega_z0 = (
    taylor_series_b.subs(N,2).doit()
    .subs([(f(a),sigma(a, g2, g3)), (f(x),sigma(x, g2, g3)), (f(x+a),sigma(x+a, g2, g3))])
    .subs([(x,z),(a,2*omega1/5)])
    .subs([d_sigma_zw_2.args, d_sigma_zw.args]).doit()
    .subs([(sigma(-2*omega1/5,g2,g3), -sigma(2*omega1/5,g2,g3)), (zw(-2*omega1/5,g2,g3), -zw(2*omega1/5,g2,g3))])
)
pw_taylor_z0 = Eq(pw(z,g2,g3), 1/z**2)
one_over_sigma_z_2z_taylor_z0 = Eq(1/sigma(z,g2,g3)/sigma(2*z,g2,g3), 1/z**2/2)
exp_eta1_a = Eq(exp(2*eta1*z), 1 + 2*eta1*z + (2*eta1*z)**2/factorial(2))
exp_eta1_b = Eq(exp(-2*eta1*z), 1 - 2*eta1*z + (2*eta1*z)**2/factorial(2))

pwp_z_UV_sig_z_0_lim_rhs_a = Eq(
    Limit(pwp_z_UV_sig_z_0_rhs_z.rhs,z,0), 
    Limit(pwp_z_UV_sig_z_0_rhs_z.rhs.subs([
        sigma_a_taylor_omega_z0.args,
        sigma_b_taylor_omega_z0.args,
        pw_taylor_z0.args,
        one_over_sigma_z_2z_taylor_z0.args,
        exp_eta1_a.args,
        exp_eta1_b.args
    ]).simplify().expand().subs(pw(-2*omega1/5,g2,g3), pw(2*omega1/5,g2,g3))
          ,z,0).doit()
)

pwp_z_UV_sig_z_0_lim = Eq(pwp_z_UV_sig_z_0_rhs_z.lhs, 
    Limit(pwp_z_UV_sig_z_0_rhs_z.rhs,z,0).subs(*pwp_z_UV_sig_z_0_lim_rhs_a.args)
    .collect(pw(2*omega1/5,g2,g3), factor)
)

pwp_z_UV_sig_z_0_rhs_z

sigma_a_taylor_omega_z0
sigma_b_taylor_omega_z0
pw_taylor_z0
one_over_sigma_z_2z_taylor_z0
exp_eta1_a
exp_eta1_b

pwp_z_UV_sig_z_0_lim_rhs_a

num_subs_simple_z0 = [(eta1, zw(omega1, g2, g3)),
 (g2, (-0.29330458302715834-0.30068856969017177j)),
 (g3, (0.4935994486020393-0.16278842953993056j)),
 (omega1, (1.7400418837920357+0.1532160102197727j)),
]
num_evaluate_weierstrass(pwp_z_UV_sig_z_0_lim, num_subs_simple_z0)

Eq(-pw(2*omega1/5, g2, g3)/2 + pwp(2*omega1/5, g2, g3)**2/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2), -pw(z, g2, g3)/2 - sigma(-2*omega1/5 + z, g2, g3)**5*exp(2*eta1*z)/(2*sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)*sigma(2*z, g2, g3)) + sigma(2*omega1/5 + z, g2, g3)**5*exp(-2*eta1*z)/(2*sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)*sigma(2*z, g2, g3)))

Eq(sigma(-2*omega1/5 + z, g2, g3), z**2*(pw(-2*omega1/5, g2, g3)*sigma(2*omega1/5, g2, g3) - sigma(2*omega1/5, g2, g3)*zw(2*omega1/5, g2, g3)**2)/2 + z*sigma(2*omega1/5, g2, g3)*zw(2*omega1/5, g2, g3) - sigma(2*omega1/5, g2, g3))

Eq(sigma(2*omega1/5 + z, g2, g3), z**2*(-pw(2*omega1/5, g2, g3)*sigma(2*omega1/5, g2, g3) + sigma(2*omega1/5, g2, g3)*zw(2*omega1/5, g2, g3)**2)/2 + z*sigma(2*omega1/5, g2, g3)*zw(2*omega1/5, g2, g3) + sigma(2*omega1/5, g2, g3))

Eq(pw(z, g2, g3), z**(-2))

Eq(1/(sigma(z, g2, g3)*sigma(2*z, g2, g3)), 1/(2*z**2))

Eq(exp(2*eta1*z), 2*eta1**2*z**2 + 2*eta1*z + 1)

Eq(exp(-2*eta1*z), 2*eta1**2*z**2 - 2*eta1*z + 1)

Eq(Limit(-pw(z, g2, g3)/2 - sigma(-2*omega1/5 + z, g2, g3)**5*exp(2*eta1*z)/(2*sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)*sigma(2*z, g2, g3)) + sigma(2*omega1/5 + z, g2, g3)**5*exp(-2*eta1*z)/(2*sigma(2*omega1/5, g2, g3)**5*sigma(z, g2, g3)*sigma(2*z, g2, g3)), z, 0, dir='+'), eta1**2 - 5*eta1*zw(2*omega1/5, g2, g3) - 5*pw(2*omega1/5, g2, g3)/4 + 25*zw(2*omega1/5, g2, g3)**2/4)

LHS 6.6229012526074 - 1.26975482442786*I
RHS 6.6229012526074 - 1.26975482442786*I
ERROR (SMAPE) 4.00575385983797e-16


Eq(-pw(2*omega1/5, g2, g3)/2 + pwp(2*omega1/5, g2, g3)**2/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2), (2*eta1 - 5*zw(2*omega1/5, g2, g3))**2/4 - 5*pw(2*omega1/5, g2, g3)/4)

In [24]:
p_duplication_b

sigma_c_pw_pwp_b = sigma_c_pw_pwp.subs(*p_duplication_b.subs(z,2*omega1/5).args).simplify()
_nn, _dd = fraction(sigma_c_pw_pwp_b.rhs)
    
sigma_c_pw_pwp_b = sigma_c_pw_pwp_b.subs(
    _dd.args[0], 
    factor(
        _dd.args[0]
        .subs(pwp(2*omega1/5,g2,g3)**2, 4*pw(2*omega1/5,g2,g3)**3 - g2*pw(2*omega1/5,g2,g3) - g3)
        .expand()
    )
  )
sigma_c_pw_pwp_b = Eq(sigma_c_pw_pwp_b.lhs, sigma_c_pw_pwp_b.rhs.factor())

sigma_c_pw_pwp_b

Eq(pw(2*z, g2, g3), (-g2/2 + 6*pw(z, g2, g3)**2)**2/(4*pwp(z, g2, g3)**2) - 2*pw(z, g2, g3))

Eq(sigma(4*omega1/5, g2, g3)**4*exp(-4*eta1*omega1/5)/(2*sigma(2*omega1/5, g2, g3)**6), 128*pwp(2*omega1/5, g2, g3)**6/(g2**2 + 24*g2*pw(2*omega1/5, g2, g3)**2 + 48*g3*pw(2*omega1/5, g2, g3) - 48*pw(2*omega1/5, g2, g3)**4)**2)

### Coefficient $c_1$

In [25]:
c0_c1_pwp2_omega1_a = Eq(c[0], 
                    solve(c_unknown_sigma.subs(z,2*omega1/5).subs(sigma(0,g2,g3),0),c[0])[0]
                    .subs(*c3_zeta.args)
                    .subs(*c2_pw_pwp.args)
                    .expand().collect(pw(2*omega1/5,g2,g3),factor) 
                   )

pw_4omega = pw_pwp_sig5_even.subs(z,4*omega1/5).subs(sigma(0,g2,g3),0)

sig_6_4_min = quasi_period.subs([(m,0),(n,1),(z,z-omega1)]).simplify().subs(z,omega1/5).subs([
    (sigma(6*omega1/5,g2,g3), -sigma(-6*omega1/5,g2,g3)),
    (sigma(-4*omega1/5,g2,g3), -sigma(4*omega1/5,g2,g3))
])
c1_sol_a = (pw_4omega.subs(*sig_6_4_min.args)
    .subs(*c3_zeta.args)
    .subs(*c2_pw_pwp.args)
    .subs(*c0_c1_pwp2_omega1_a.args)
)

c1_sol_b = Eq(
    (c1_sol_a.lhs + pwp(2*omega1/5, g2, g3)**3/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2).factor(), 
              c1_sol_a.rhs + pwp(2*omega1/5, g2, g3)**3/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)

pw_2_4_omega = (
    sigma_p_identity
    .subs(y,z)
    .subs(x, 2*omega1/5)
    .subs(z,4*omega1/5)
    .subs(
        sigma(-2*omega1/5,g2,g3), -sigma(2*omega1/5,g2,g3)
    )
    .subs(
        *(quasi_period.subs([(m,0),(n,1),(z,z-omega1)]).simplify().subs(z,omega1/5)
        .subs(sigma(-4*omega1/5,g2,g3), -sigma(4*omega1/5,g2,g3))).args
    )
)

pwp_2_4_omega = pwp_sigma_dbl_ratio.subs(z,2*omega1/5)

pw_pwp_2_4_omega = Eq(pwp_2_4_omega.rhs*pw_2_4_omega.rhs, pwp_2_4_omega.lhs*pw_2_4_omega.lhs)

pw_pwp_sig_4_5_omega = (pwp_2_4_omega
                        .subs(*Eq(sigma(2*omega1/5,g2,g3), 
                                  solve(pw_2_4_omega, sigma(2*omega1/5,g2,g3))[0]).args)
                       )
pw_pwp_sig_4_5_omega = Eq(-(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**4/pw_pwp_sig_4_5_omega.rhs, 
                          -(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**4/pw_pwp_sig_4_5_omega.lhs)

c1_sol_c = c1_sol_b.subs([pw_pwp_2_4_omega.args, pw_pwp_sig_4_5_omega.args])
c1_sol_d = Eq(c1_sol_c.lhs/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))/2,
             (c1_sol_c.rhs/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))/2
             ).expand().collect(pwp(2*omega1/5,g2,g3),factor)
             )
c1_sol_d = Eq( c[1], -c1_sol_d.rhs + factor(c1_sol_d.lhs+c[1]))

# pw_2_4_omega
# pwp_2_4_omega
# pw_pwp_2_4_omega
# pw_pwp_sig_4_5_omega
# pw_4omega
# c1_sol_a
# c1_sol_b  
# c1_sol_c
c1_sol_d

Eq(c[1], (2*eta1 - 5*zw(2*omega1/5, g2, g3))*(pw(2*omega1/5, g2, g3) + pw(4*omega1/5, g2, g3)) + (pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3/(2*pwp(2*omega1/5, g2, g3)) + pwp(2*omega1/5, g2, g3)/2 - pwp(2*omega1/5, g2, g3)**3/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3))

### Coefficient $c_0$

In [26]:
c0_pwp2_omega1_b = c0_c1_pwp2_omega1_a.subs(*c1_sol_d.args)
c0_pwp2_omega1_c = Eq(c0_pwp2_omega1_b.lhs, c0_pwp2_omega1_b.rhs.expand().collect(pwp(2*omega1/5,g2,g3),factor))

# c0_pwp2_omega1_a
# c0_pwp2_omega1_b
c0_pwp2_omega1_c

Eq(c[0], -(2*eta1 - 5*zw(2*omega1/5, g2, g3))*pw(2*omega1/5, g2, g3)*pw(4*omega1/5, g2, g3) - (pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3*pw(2*omega1/5, g2, g3)/(2*pwp(2*omega1/5, g2, g3)) - pw(2*omega1/5, g2, g3)*pwp(2*omega1/5, g2, g3)/2 + pw(4*omega1/5, g2, g3)*pwp(2*omega1/5, g2, g3)**3/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3))

## System 1: The system solved by Kronecker theta functions

Two systems will be considered that conserve the quantity that stems from the curve generated by the Frobenius Stickelberger determinant at 5th order, the quantity that is considered herein for generating Hamiltonians. 

System 1 normalises the curve such that the derivative of the product of the two modes is the Weierstrass P function. It will be shown then that System 1 can be solved in terms of Kronecker theta functions. It will be shown that the system can be derived in two ways which we refer to as Method 1 and Method 2. Method 1 uses the defining differential equation of the Kronecker theta function and an expression for Weierstrass P in terms of the modes as a coupling term. Method 2 uses a Hamiltonian based on the normalised Frobenius Sticklberger determinant to derive the system. The study of System 1 is complete and agreement with numerical solutions is demonstrated.

A word of caution in System 1, we should be aware that by fixing the value of the constant term, or rather the value of the Hamiltonian, we are describing a system in which we have given up some freedom in the initial conditions. If the system parameters are fixed and the value of the Hamiltonian is fixed, then if we pick say $U(0)$ then we lose the freedom to arbitrarily pick $V(0)$ as its value must already be decided by the fixed Hamiltonian. This was not an issue at lower order systems where we were free to pick arbitrary launch conditions and still solve in terms of elliptic functions. When we construct a Hamiltonian set to a non-fixed value it yields hyperelliptic functions but this is not explored further herein.


In System 2 it is shown that the simpler polynomial Hamiltonian without normalisation leads to pseudo-hyperelliptic functions, albeit with a repeated root. This is a system of considerable interest and it is intimately related to System 1 and possibly related to the Carlson symmetric $R_j$ elliptic integral. The study of System 2 is ongoing and the complete solutions are not yet obtained, although some insights are obtained.

### Method 1: Deriving the dynamic system from the differential equation for the Kronecker theta function and coupling via Weierstrass P Prime (no Hamiltonian method)

By deriving expressions for the even and odd parts of the determinant identity, it is possible to isolate an expression for Weierstrass P Prime in terms of $U(z),V(z)$. This expression can be used as a substitution in the general expression for the logarithmic derivative of a Kronecker theta function to build a coupled system. In general, the logarithmic derivative of the Kronecker theta function is given by:

In [27]:
log_kron_diff = Eq(Derivative(log(sigma(x + z, g2, g3)/(sigma(x, g2, g3)*sigma(z, g2, g3))), z),
 zw(x, g2, g3) + (pwp(x, g2, g3) - pwp(z, g2, g3))/(2*(pw(x, g2, g3) - pw(z, g2, g3)))
  )

# Note: the minus sign is ignored inside the log as the derivative would kill the 2*I*pi anyway
log_kron_diff_conj = log_kron_diff.subs(x,-x).subs([
    (pwp(-x,g2,g3), -pwp(x,g2,g3)), (pw(-x,g2,g3), pw(x,g2,g3)), (zw(-x,g2,g3), -zw(x,g2,g3))
]).subs(sigma(-x,g2,g3), sigma(x,g2,g3))

log_diff_kron2 = Eq(
    Derivative(log(sigma(x + z, g2, g3)/(sigma(x, g2, g3)*sigma(z, g2, g3))*exp(-zw(x, g2, g3)*z)), z),
    (pwp(x, g2, g3) - pwp(z, g2, g3))/(2*pw(x, g2, g3) - 2*pw(z, g2, g3))
)

log_diff_kron2_conj = Eq(
    Derivative(log(sigma(-x + z, g2, g3)/(sigma(x, g2, g3)*sigma(z, g2, g3))*exp(zw(x, g2, g3)*z)), z),
    (-pwp(x, g2, g3) - pwp(z, g2, g3))/(2*pw(x, g2, g3) - 2*pw(z, g2, g3))
)

log_kron_diff
log_kron_diff_conj

log_diff_kron2
log_diff_kron2_conj

Eq(Derivative(log(sigma(x + z, g2, g3)/(sigma(x, g2, g3)*sigma(z, g2, g3))), z), zw(x, g2, g3) + (pwp(x, g2, g3) - pwp(z, g2, g3))/(2*pw(x, g2, g3) - 2*pw(z, g2, g3)))

Eq(Derivative(log(sigma(-x + z, g2, g3)/(sigma(x, g2, g3)*sigma(z, g2, g3))), z), -zw(x, g2, g3) + (-pwp(x, g2, g3) - pwp(z, g2, g3))/(2*pw(x, g2, g3) - 2*pw(z, g2, g3)))

Eq(Derivative(log(sigma(x + z, g2, g3)*exp(-z*zw(x, g2, g3))/(sigma(x, g2, g3)*sigma(z, g2, g3))), z), (pwp(x, g2, g3) - pwp(z, g2, g3))/(2*pw(x, g2, g3) - 2*pw(z, g2, g3)))

Eq(Derivative(log(sigma(-x + z, g2, g3)*exp(z*zw(x, g2, g3))/(sigma(x, g2, g3)*sigma(z, g2, g3))), z), (-pwp(x, g2, g3) - pwp(z, g2, g3))/(2*pw(x, g2, g3) - 2*pw(z, g2, g3)))

By equating the odd parts of the determinant expression it can be shown that:

In [28]:
sigma_Uz_sub = Eq(sigma(-2*omega1/5 + z, g2, g3), solve(Uz_sig,sigma(-2*omega1/5 + z, g2, g3))[0])
sigma_Vz_sub = Eq(sigma(2*omega1/5 + z, g2, g3), solve(Vz_sig,sigma(2*omega1/5 + z, g2, g3))[0])

kappa_UV = Eq(kappa, pwp(2*omega1/5, g2, g3)**2/U(4*omega1/5)**2/V(4*omega1/5)**2)
kappa_pw = kappa_UV.subs(*UVz_pw.subs(z,4*omega1/5).args)

UV_pwp_z = pwp_z_UV_sig.subs([
    sigma_Uz_sub.args, 
    sigma_Vz_sub.args, 
    (pw(z,g2,g3), solve(UVz_pw, pw(z,g2,g3))[0]),
    (pwp(2*omega1/5, g2, g3)**2, solve(kappa_pw, pwp(2*omega1/5, g2, g3)**2)[0])
]).expand()
UV_pwp_z = Eq(UV_pwp_z.lhs*2, UV_pwp_z.rhs*2)
pwp_z_UV = Eq(pwp(z,g2,g3), (solve(UV_pwp_z, pwp(z,g2,g3))[0]).collect([U(z), V(z)]))

kappa_UV
kappa_pw
UV_pwp_z
pwp_z_UV

Eq(kappa, pwp(2*omega1/5, g2, g3)**2/(U(4*omega1/5)**2*V(4*omega1/5)**2))

Eq(kappa, pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)

Eq(-U(z)*V(z)*pwp(z, g2, g3) + pwp(z, g2, g3)*kappa, U(z)**5 - V(z)**5)

Eq(pwp(z, g2, g3), (-U(z)**5 + V(z)**5)/(U(z)*V(z) - kappa))

By inserting that expression for Weierstrass P Prime into the the logarithmic derivative of the Kronecker theta function, the following coupled system is obtained:

In [29]:
log_kron_diff_U = log_kron_diff_conj.subs(x,2*omega1/5).subs([
    sigma_Uz_sub.args, 
    (2*UVz_pw.rhs, 2*UVz_pw.lhs),
    pwp_z_UV.args
]).doit().expand()
log_kron_diff_U = Eq((log_kron_diff_U.lhs + 2*eta1/5)*U(z), ((log_kron_diff_U.rhs + 2*eta1/5)*U(z)).expand())

log_kron_diff_V = log_kron_diff.subs(x,2*omega1/5).subs([
    sigma_Vz_sub.args, 
    (2*UVz_pw.rhs, 2*UVz_pw.lhs),
    pwp_z_UV.args
]).doit().expand()
log_kron_diff_V = Eq((log_kron_diff_V.lhs - 2*eta1/5)*V(z), ((log_kron_diff_V.rhs - 2*eta1/5)*V(z)).expand())

dUV_U5_V5 = Eq(
    log_kron_diff_U.lhs*V(z) + log_kron_diff_V.lhs*U(z),
    ((log_kron_diff_U.rhs*V(z) + log_kron_diff_V.rhs*U(z)).expand()).simplify()
)

log_kron_diff_U
log_kron_diff_V
dUV_U5_V5

UVz_pw

Eq(
    log_kron_diff_U.lhs*V(z) + log_kron_diff_V.lhs*U(z),
    ((log_kron_diff_U.rhs*V(z) + log_kron_diff_V.rhs*U(z)).expand() + pwp_z_UV.rhs - pwp_z_UV.lhs).simplify()
)

Eq(Derivative(U(z), z), 2*eta1*U(z)/5 - U(z)*zw(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3)/(2*V(z)) - U(z)*V(z)**4/(2*U(z)**2*V(z) - 2*U(z)*kappa) + U(z)**5/(2*U(z)*V(z)**2 - 2*V(z)*kappa))

Eq(Derivative(V(z), z), -2*eta1*V(z)/5 + V(z)*zw(2*omega1/5, g2, g3) + pwp(2*omega1/5, g2, g3)/(2*U(z)) - V(z)**5/(2*U(z)**2*V(z) - 2*U(z)*kappa) + U(z)**4*V(z)/(2*U(z)*V(z)**2 - 2*V(z)*kappa))

Eq(U(z)*Derivative(V(z), z) + V(z)*Derivative(U(z), z), (U(z)**5 - V(z)**5)/(U(z)*V(z) - kappa))

Eq(U(z)*V(z), pw(2*omega1/5, g2, g3) - pw(z, g2, g3))

Eq(U(z)*Derivative(V(z), z) + V(z)*Derivative(U(z), z), -pwp(z, g2, g3))

#### Simplifying the coefficients to express them as rational functions of just two parameters

By isolating an expression involving only even functions of $z$, it is possible to replace $\mathrm{pw}(z,g_2,g_3)$ with $U(z),V(z)$.

In [30]:
pw_pwp_sig5_even = Eq(c_unknown_sigma.lhs + c_unknown_sigma.lhs.subs(z,-z), 
   c_unknown_sigma.rhs + c_unknown_sigma.rhs.subs(z,-z)).subs(
    [(pw(-z,g2,g3),pw(z,g2,g3)), (pwp(-z,g2,g3),-pwp(z,g2,g3)), (sigma(-z,g2,g3),-sigma(z,g2,g3))])

pw_pwp_sig5_even_UV = (pw_pwp_sig5_even
                       .subs(sigma(-2*omega1/5-z,g2,g3),-sigma(2*omega1/5+z,g2,g3))
                       .subs([(Uz_sig.rhs, Uz_sig.lhs),(Vz_sig.rhs, Vz_sig.lhs)])
                       .subs(pw(z,g2,g3), solve(UVz_pw,pw(z,g2,g3))[0])
                      )
pw_pwp_sig5_even_UV = Eq(pw_pwp_sig5_even_UV.lhs.expand().collect(U(z)*V(z)), pw_pwp_sig5_even_UV.rhs)

d3_c_UV = Eq(pw_pwp_sig5_even_UV.lhs.coeff(U(z)*V(z),2),d[3])
d1_c_UV = Eq(pw_pwp_sig5_even_UV.lhs.coeff(U(z)*V(z),1),d[1])
d0_c_UV = Eq(pw_pwp_sig5_even_UV.lhs.subs(U(z)*V(z),0), d[0])

pw_pwp_sig5_even_UV_d = pw_pwp_sig5_even_UV.subs([d0_c_UV.args, d1_c_UV.args, d3_c_UV.args])
pw_pwp_sig5_even_UV_d = Eq(d[0], pw_pwp_sig5_even_UV_d.rhs - pw_pwp_sig5_even_UV_d.lhs + d[0])

d3_pw_pwp = Eq(d3_c_UV.rhs, d3_c_UV.lhs.subs(*c3_zeta.args))
d1_pw_pwp = Eq(d1_c_UV.rhs, 
   d1_c_UV.subs([c1_sol_d.args, c3_zeta.args]).lhs
   .expand().collect(pwp(2*omega1/5,g2,g3),factor)
  )
d0_pw_pwp = Eq(d0_c_UV.rhs, 
   d0_c_UV.subs([c1_sol_d.args, c3_zeta.args, c0_pwp2_omega1_c.args]).lhs
   .expand().collect(pwp(2*omega1/5,g2,g3),factor)
  )


UVz_pw
d3_c_UV
d1_c_UV
d0_c_UV

d0_pw_pwp
d1_pw_pwp
d3_pw_pwp

pw_pwp_sig5_even_UV_d

Eq(U(z)*V(z), pw(2*omega1/5, g2, g3) - pw(z, g2, g3))

Eq(2*c[3], d[3])

Eq(-4*pw(2*omega1/5, g2, g3)*c[3] - 2*c[1], d[1])

Eq(2*pw(2*omega1/5, g2, g3)**2*c[3] + 2*pw(2*omega1/5, g2, g3)*c[1] + 2*c[0], d[0])

Eq(d[0], -pwp(2*omega1/5, g2, g3)**3/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)

Eq(d[1], 2*(2*eta1 - 5*zw(2*omega1/5, g2, g3))*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3)) - (pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3/pwp(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3) + pwp(2*omega1/5, g2, g3)**3/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3)

Eq(d[3], -4*eta1 + 10*zw(2*omega1/5, g2, g3))

Eq(d[0], U(z)**5 - U(z)**2*V(z)**2*d[3] - U(z)*V(z)*d[1] + V(z)**5)

By differentiating the expression for $d_0$ w.r.t $z$ and then using the differential equations for $U(z),V(z)$ derived earlier, it is possible to show the following interesting identity relating the Weierstrass zeta terms to Weierstrass P and P Prime:

In [31]:
_nn,_dd = fraction(diff(pw_pwp_sig5_even_UV_d.rhs,z).subs([log_kron_diff_U.args, log_kron_diff_V.args]).simplify())


_cvbnc = (
    factor(_nn.expand()).args[2].expand().collect([U(z),V(z)],factor) 
    - pw_pwp_sig5_even_UV_d.rhs*5 + pw_pwp_sig5_even_UV_d.lhs*5
).expand().collect([U(z),V(z)],factor).subs([d0_pw_pwp.args, kappa_pw.args, d3_pw_pwp.args, d1_pw_pwp.args])



_ZXY = Eq(Z, solve(_cvbnc.expand().collect(pwp(2*omega1/5,g2,g3),factor).subs(U(z)*V(z),1).subs([
    (pwp(2*omega1/5,g2,g3), X),(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3), Y), 
    (2*eta1 - 5*zw(2*omega1/5,g2,g3), Z)
]),Z)[0]
  )

eta_zw_pw_pwp_2_5 = _ZXY.subs([(X,pwp(2*omega1/5,g2,g3)),(Y,pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3)), 
    (Z,2*eta1 - 5*zw(2*omega1/5,g2,g3))])

pw_pwp_sig5_even_UV_d
_nn.expand().collect([U(z),V(z)],factor)
_cvbnc
_ZXY
eta_zw_pw_pwp_2_5
log_kron_diff_U
log_kron_diff_V
num_evaluate_weierstrass(eta_zw_pw_pwp_2_5, num_subs_simple)

Eq(d[0], U(z)**5 - U(z)**2*V(z)**2*d[3] - U(z)*V(z)*d[1] + V(z)**5)

-2*(-2*eta1 + 5*zw(2*omega1/5, g2, g3) + 2*d[3])*U(z)**7*V(z)**2 + 2*(-2*eta1 + 5*zw(2*omega1/5, g2, g3) + 2*d[3])*U(z)**2*V(z)**7 - (4*eta1*kappa + 5*pwp(2*omega1/5, g2, g3) - 10*zw(2*omega1/5, g2, g3)*kappa + 2*d[1])*U(z)**6*V(z) + (4*eta1*kappa + 5*pwp(2*omega1/5, g2, g3) - 10*zw(2*omega1/5, g2, g3)*kappa + 2*d[1])*U(z)*V(z)**6 + 5*U(z)**10 + 5*U(z)**5*pwp(2*omega1/5, g2, g3)*kappa - 5*V(z)**10 - 5*V(z)**5*pwp(2*omega1/5, g2, g3)*kappa

(-4*eta1*pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2 + 6*(2*eta1 - 5*zw(2*omega1/5, g2, g3))*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3)) - 3*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3/pwp(2*omega1/5, g2, g3) - 8*pwp(2*omega1/5, g2, g3) + 10*pwp(2*omega1/5, g2, g3)**2*zw(2*omega1/5, g2, g3)/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2 + 3*pwp(2*omega1/5, g2, g3)**3/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3)*U(z)*V(z)

Eq(Z, (3*X**2 + Y**3)/(2*X*Y))

Eq(2*eta1 - 5*zw(2*omega1/5, g2, g3), ((pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3 + 3*pwp(2*omega1/5, g2, g3)**2)/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))*pwp(2*omega1/5, g2, g3)))

Eq(Derivative(U(z), z), 2*eta1*U(z)/5 - U(z)*zw(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3)/(2*V(z)) - U(z)*V(z)**4/(2*U(z)**2*V(z) - 2*U(z)*kappa) + U(z)**5/(2*U(z)*V(z)**2 - 2*V(z)*kappa))

Eq(Derivative(V(z), z), -2*eta1*V(z)/5 + V(z)*zw(2*omega1/5, g2, g3) + pwp(2*omega1/5, g2, g3)/(2*U(z)) - V(z)**5/(2*U(z)**2*V(z) - 2*U(z)*kappa) + U(z)**4*V(z)/(2*U(z)*V(z)**2 - 2*V(z)*kappa))

LHS -5.5554021279856 - 1.81089400706177*I
RHS -5.5554021279856 - 1.81089400706177*I
ERROR (SMAPE) 0


Eq(2*eta1 - 5*zw(2*omega1/5, g2, g3), ((pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3 + 3*pwp(2*omega1/5, g2, g3)**2)/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))*pwp(2*omega1/5, g2, g3)))

Consequently, it turns out all coefficients can be expressed as rational functions of two parameters $X,Y$:

In [32]:
d1_pw_pwp_b = d1_pw_pwp.subs(*eta_zw_pw_pwp_2_5.args).simplify()
d3_pw_pwp_b = d3_pw_pwp.subs(eta_zw_pw_pwp_2_5.lhs*2, eta_zw_pw_pwp_2_5.rhs*2)

d0_pw_pwp
d1_pw_pwp_b
d3_pw_pwp_b
c2_pw_pwp
kappa_pw

pwp_X_5 = Eq(pwp(2*omega1/5,g2,g3), X)
pw_X_245 = Eq(pw(2*omega1/5,g2,g3) - pw(4*omega1/5,g2,g3), Y)
UV_ham_XY_poly = (pw_pwp_sig5_even_UV_d
    .subs([d0_pw_pwp.args, d1_pw_pwp_b.args, d3_pw_pwp_b.args, kappa_pw.args])
    .subs([pwp_X_5.args, pw_X_245.args])
                 )

pwp_X_5
pw_X_245
UV_ham_XY_poly

Eq(d[0], -pwp(2*omega1/5, g2, g3)**3/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)

Eq(d[1], 2*pwp(2*omega1/5, g2, g3) + pwp(2*omega1/5, g2, g3)**3/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3)

Eq(d[3], -((pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3 + 3*pwp(2*omega1/5, g2, g3)**2)/((pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))*pwp(2*omega1/5, g2, g3)))

Eq(c[2], -pw(2*omega1/5, g2, g3)/2 + pwp(2*omega1/5, g2, g3)**2/(2*(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2))

Eq(kappa, pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)

Eq(pwp(2*omega1/5, g2, g3), X)

Eq(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3), Y)

Eq(-X**3/Y**2, -(X**3/Y**3 + 2*X)*U(z)*V(z) + U(z)**5 + V(z)**5 + (3*X**2 + Y**3)*U(z)**2*V(z)**2/(X*Y))

In [33]:
dUV_pwp = Eq(diff(UVz_pw.lhs,z), diff(UVz_pw.rhs,z).subs(diff(pw(z,g2,g3),z),pwp(z,g2,g3)))
dUV_pwp_b = Eq(dUV_U5_V5.rhs, dUV_U5_V5.lhs).subs(*dUV_pwp.args)
g2_g3_UV_a = Eq(dUV_U5_V5.rhs**2*(U(z)*V(z)-kappa)**2 - 
   (4*pw(z,g2,g3)**3 - g2*pw(z,g2,g3) - g3).subs(pw(z,g2,g3), solve(UVz_pw, pw(z,g2,g3))[0])*(U(z)*V(z)-kappa)**2
   ,0)

UV5_K = Eq(-pw_pwp_sig5_even_UV_d.rhs + d[0] + U(z)**5 + V(z)**5, 
           -pw_pwp_sig5_even_UV_d.lhs + d[0] + U(z)**5 + V(z)**5)

g2_g3_UV_b = Eq((g2_g3_UV_a.lhs - UV5_K.rhs**2 + UV5_K.lhs**2).expand().collect(U(z)*V(z),factor),0)

uvz4_coeff = Eq(g2_g3_UV_b.lhs.coeff(U(z)*V(z),4).subs([d3_pw_pwp_b.args, kappa_pw.args]),0)
pwXY25 = Eq(pw(2*omega1/5,g2,g3), solve(uvz4_coeff.subs([pwp_X_5.args, pw_X_245.args]), pw(2*omega1/5,g2,g3))[0])


g2XY = Eq(g2, 
   g2 + 
   g2_g3_UV_b.lhs
   .coeff(U(z)*V(z),3)
   .subs([d3_pw_pwp_b.args, d1_pw_pwp_b.args, kappa_pw.args])
   .subs([pwp_X_5.args, pw_X_245.args])
   .subs(*pwXY25.args)
   .simplify()
  )

g3XY = Eq(pwp_X_5.lhs**2 - X**2,0).subs(
    pwp(2*omega1/5,g2,g3)**2, 4*pw(2*omega1/5,g2,g3)**3 -g2*pw(2*omega1/5,g2,g3)-g3
).subs(*pwXY25.args).subs(*g2XY.args)
g3XY = Eq(g3, solve(g3XY.lhs,g3)[0].expand())

pwXY45 = Eq(p_duplication_c.lhs.subs(z,2*omega1/5), 
   p_duplication_c.rhs
   .subs(z,2*omega1/5)
   .subs(*pwp_X_5.args)
   .subs([pwXY25.args, g2XY.args, g3XY.args])
   .simplify()
  )

dUV_U5_V5
UVz_pw
dUV_pwp
dUV_pwp_b
g2_g3_UV_a
UV5_K
g2_g3_UV_b
num_evaluate_weierstrass(uvz4_coeff, num_subs_simple)
pwXY25
pwXY45
g2XY
g3XY

Eq(U(z)*Derivative(V(z), z) + V(z)*Derivative(U(z), z), (U(z)**5 - V(z)**5)/(U(z)*V(z) - kappa))

Eq(U(z)*V(z), pw(2*omega1/5, g2, g3) - pw(z, g2, g3))

Eq(U(z)*Derivative(V(z), z) + V(z)*Derivative(U(z), z), -pwp(z, g2, g3))

Eq((U(z)**5 - V(z)**5)/(U(z)*V(z) - kappa), -pwp(z, g2, g3))

Eq(-(U(z)*V(z) - kappa)**2*(-g2*(-U(z)*V(z) + pw(2*omega1/5, g2, g3)) - g3 + 4*(-U(z)*V(z) + pw(2*omega1/5, g2, g3))**3) + (U(z)**5 - V(z)**5)**2, 0)

...

Eq(((pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**3 + 3*pwp(2*omega1/5, g2, g3)**2)**2/((pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2*pwp(2*omega1/5, g2, g3)**2) - 12*pw(2*omega1/5, g2, g3) - 8*pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2, 0)

Eq(pw(2*omega1/5, g2, g3), (-8*X**4 + (3*X**2 + Y**3)**2)/(12*X**2*Y**2))

Eq(pw(4*omega1/5, g2, g3), X**2/(12*Y**2) - Y/2 + Y**4/(12*X**2))

Eq(g2, X**4/(12*Y**4) - X**2/Y + 7*Y**2/6 + Y**5/X**2 + Y**8/(12*X**4))

Eq(g3, -X**6/(216*Y**6) + X**4/(12*Y**3) - 25*X**2/72 - 25*Y**6/(72*X**2) - Y**9/(12*X**4) - Y**12/(216*X**6))

In [34]:
pwx_2_4 = Eq(pw(x,g2,g3) , pw(z,g2,g3)+(U(z)*V(z) - kappa).subs(*UVz_pw.args).subs(*kappa_pw.args))
pwx_2_4_XY = pwx_2_4.subs([pwXY25.args, pwXY45.args, pwp_X_5.args]).simplify()
pwx_2_4_XY_b = pwx_2_4.subs(*pwx_2_4_XY.args)

pwx_2_4
pwx_2_4_XY
pwx_2_4_XY_b
pwXY25.expand()

Eq(pw(x, g2, g3), pw(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)

Eq(pw(x, g2, g3), -11*X**2/(12*Y**2) + Y/2 + Y**4/(12*X**2))

Eq(-11*X**2/(12*Y**2) + Y/2 + Y**4/(12*X**2), pw(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)

Eq(pw(2*omega1/5, g2, g3), X**2/(12*Y**2) + Y/2 + Y**4/(12*X**2))

#### Attempting to solve for $X,Y$ in terms of $g_2,g_3$
In theory, there is an algebraic relation between $X,Y$ and $g_2,g_3$ but this appears to involve solving a high order polynomial.

In [35]:

discrim_g_2_3 = Eq(g2**3 - 27*g3**2, (g2**3 - 27*g3**2).subs([g2XY.args,g3XY.args]).expand())
discrim_g_2_3_delta = Eq(discrim_g_2_3.lhs, Delta)
x_discrim = discrim_g_2_3.subs([discrim_g_2_3_delta.args, (X**2,x)])
x_discrim_sols = solve(x_discrim,x)
X_discrim_Y_a = Eq(X**2, x_discrim_sols[0])
X_discrim_Y_b =Eq(X**2, x_discrim_sols[1])
X_discrim_Y_a_one_over = Eq(1/X_discrim_Y_a.lhs, 
                            (X_discrim_Y_b.rhs/(X_discrim_Y_a.rhs*X_discrim_Y_b.rhs).expand()))

discrim_g_2_3_delta
discrim_g_2_3
X_discrim_Y_a
X_discrim_Y_b
X_discrim_Y_a_one_over

Eq(g2**3 - 27*g3**2, Delta)

Eq(g2**3 - 27*g3**2, X**2*Y**3 - 11*Y**6 - Y**9/X**2)

Eq(X**2, (Delta + 11*Y**6 - sqrt(Delta**2 + 22*Delta*Y**6 + 125*Y**12))/(2*Y**3))

Eq(X**2, (Delta + 11*Y**6 + sqrt(Delta**2 + 22*Delta*Y**6 + 125*Y**12))/(2*Y**3))

Eq(X**(-2), -(Delta + 11*Y**6 + sqrt(Delta**2 + 22*Delta*Y**6 + 125*Y**12))/(2*Y**9))

In [36]:
g2_Y = Eq(g2, 
   (X**4/(12*Y**4) - X**2/Y + 7*Y**2/6).subs(*X_discrim_Y_a.args) + 
   (Y**5/X**2 + Y**8/(12*X**4)).subs(*X_discrim_Y_a_one_over.args)
).expand()

g3_Y = Eq(g3,
    (-X**6/(216*Y**6) + X**4/(12*Y**3) - 25*X**2/72).subs(*X_discrim_Y_a.args) + 
    (-25*Y**6/(72*X**2) - Y**9/(12*X**4) - Y**12/(216*X**6)).subs(*X_discrim_Y_a_one_over.args)
).expand()

g2_Y
g3_Y

Eq(g2, Delta**2/(12*Y**10) + 5*Delta/(6*Y**4) + 5*Y**2/12)

Eq(g3, Delta**2*sqrt(Delta**2 + 22*Delta*Y**6 + 125*Y**12)/(216*Y**15) + Delta*sqrt(Delta**2 + 22*Delta*Y**6 + 125*Y**12)/(54*Y**9) - sqrt(Delta**2 + 22*Delta*Y**6 + 125*Y**12)/(216*Y**3))

In [37]:
Xxy32 = Eq(X,x*Y**Rational(3,2))
DeltaYy = Eq(Delta,(2*y-11)*Y**6)

Xxy32
DeltaYy
X_discrim_Y_a.subs([Xxy32.args,]).simplify()

Eq(X, Y**(3/2)*x)

Eq(Delta, Y**6*(2*y - 11))

Eq(Y**3*x**2, (Delta + 11*Y**6 - sqrt(Delta**2 + 22*Delta*Y**6 + 125*Y**12))/(2*Y**3))

In [38]:
xy_sqrt = Eq(x**2, y - sqrt(y**2+1))
yt_sub = Eq(y,(t - 1/t)/2)
xsqrd_one_over_t = xy_sqrt.subs(*yt_sub.args).simplify().subs(sqrt(t**2 + 2 + 1/t**2),t + 1/t).simplify()
yt_sub_b = yt_sub.subs(t, solve(xsqrd_one_over_t,t)[0])
xy_sqrt
yt_sub
xsqrd_one_over_t
yt_sub_b

Eq(x**2, y - sqrt(y**2 + 1))

Eq(y, t/2 - 1/(2*t))

Eq(1/t, -x**2)

Eq(y, x**2/2 - 1/(2*x**2))

In [39]:
yt_sub_b.subs(x, solve(Xxy32,x)[0])

Eq(y, X**2/(2*Y**3) - Y**3/(2*X**2))

In [40]:
discrim_g_2_3.subs(Xxy32.lhs**2, Xxy32.rhs**2).subs(Y**6, solve(DeltaYy,Y**6)[0])

Eq(g2**3 - 27*g3**2, Delta*x**2/(2*y - 11) - 11*Delta/(2*y - 11) - Delta/(x**2*(2*y - 11)))

### Method 2: Elliptic Hamiltonian

In this example, the polynomial from the previous section is divided by the terms that were multiplying the Weierstrass P Prime function to give a new Hamiltonian system that is solved by Kronecker theta functions. The starting Hamiltonian for the system solved by Kronecker theta functions is:

In [41]:
K_UV_ellip = Eq(
    ((pw_pwp_sig5_even_UV_d.lhs - pw_pwp_sig5_even_UV_d.rhs)/(U(z)*V(z)-kappa)/5)
    .expand()
    .subs([d0_pw_pwp.args, d1_pw_pwp_b.args, d3_pw_pwp_b.args, kappa_pw.args])
    .subs([pwp_X_5.args, pw_X_245.args])
    .simplify().expand().collect([U(z),V(z)],simplify)
        ,0)

K_UV_ellip_apart = Eq(
    (apart(K_UV_ellip.lhs.subs([(U(z)**5,0), (V(z)**5,0)]).subs(U(z),Z/V(z)), Z) + 
    (K_UV_ellip.lhs - K_UV_ellip.lhs.subs([(U(z)**5,0), (V(z)**5,0)])))
    .subs(Z,U(z)*V(z)),
    K_UV_ellip.rhs
)
non_UV_part = sum(t for t in K_UV_ellip_apart.lhs.args if x not in t.subs([(U(z),x),(V(z),x)]).free_symbols)
K_UV_ellip_b = Eq(K_UV_ellip_apart.lhs - non_UV_part, K_UV_ellip_apart.rhs - non_UV_part)

non_UV5_part = sum(t for t in K_UV_ellip_b.lhs.args if x not in t.subs([(U(z)**5,x),(V(z)**5,x)]).free_symbols)
K_UV_ellip_b2 = Eq(K_UV_ellip_b.lhs - non_UV5_part, K_UV_ellip_b.rhs - non_UV5_part)

_UV_term__ = (3*X**2 + Y**3)/(5*X*Y)*U(z)*V(z)
_UV_term_denom = -X**2 + Y**2*U(z)*V(z)
K_UV_ellip_b3 = Eq(
    sum((Y**2*t/_UV_term_denom).factor() for t in  (K_UV_ellip_b.lhs + _UV_term__).args), 
    sum(apart((Y**2*t/_UV_term_denom).subs(U(z)*V(z),Z),Z).subs(Z,U(z)*V(z)) 
        for t in  (K_UV_ellip_b.rhs + _UV_term__).args)
)

# K_UV_ellip
# K_UV_ellip_apart
K_UV_ellip_b
# K_UV_ellip_b2
K_UV_ellip_b3

Eq(2*X**5/(5*Y**3*(X**2 - Y**2*U(z)*V(z))) + Y**2*U(z)**5/(5*(X**2 - Y**2*U(z)*V(z))) + Y**2*V(z)**5/(5*(X**2 - Y**2*U(z)*V(z))) - (3*X**2 + Y**3)*U(z)*V(z)/(5*X*Y), (2*X**3 - X*Y**3)/(5*Y**3))

Eq(-2*X**5/(5*Y*(-X**2 + Y**2*U(z)*V(z))**2) - Y**4*U(z)**5/(5*(-X**2 + Y**2*U(z)*V(z))**2) - Y**4*V(z)**5/(5*(-X**2 + Y**2*U(z)*V(z))**2), -X*(2*X**2 - Y**3)/(5*Y*(X**2 - Y**2*U(z)*V(z))) - X*(3*X**2 + Y**3)/(5*Y*(X**2 - Y**2*U(z)*V(z))) + (3*X**2 + Y**3)/(5*X*Y))

The system of differential equations and their solutions are:

In [42]:
dU_ellip = Eq(diff(U(z),z), sum(t.factor() for t in diff(K_UV_ellip_b.lhs, V(z)).args))
dV_ellip = Eq(diff(V(z),z), -sum(t.factor() for t in diff(K_UV_ellip_b.lhs, U(z)).args))
dU_ellip_b = dU_ellip.subs(sum(t*U(z) for t in K_UV_ellip_b3.lhs.args), 
                           sum(t*U(z) for t in K_UV_ellip_b3.rhs.args))
dU_ellip_b = Eq(dU_ellip_b.lhs, 
                sum(t.factor() for t in dU_ellip_b.rhs.collect([U(z)/(-_UV_term_denom)/(5*Y)]).args)
                   )
dV_ellip_b = dV_ellip.subs(-sum(t*V(z) for t in K_UV_ellip_b3.lhs.args), 
                           -sum(t*V(z) for t in K_UV_ellip_b3.rhs.args))
dV_ellip_b = Eq(dV_ellip_b.lhs, 
                sum(t.factor() for t in dV_ellip_b.rhs.collect([V(z)/(-_UV_term_denom)/(5*Y)]).args)
                   )
dUV_ellip = Eq(Derivative(U(z)*V(z),z), 
               Derivative(U(z)*V(z),z).doit().subs([dU_ellip_b.args, dV_ellip_b.args]).simplify())

dUV_ellip_sqrd = Eq(dUV_ellip.lhs**2, 
   (dUV_ellip.rhs**2 - (5*K_UV_ellip_b2.lhs)**2 + (5*K_UV_ellip_b2.rhs)**2)
   .simplify().factor().expand().collect(U(z)*V(z), factor)
  )

W_alpha_UV = Eq(U(z)*V(z), -W(z)+alpha)
dW_alpha = Eq(
    dUV_ellip_sqrd.lhs.subs(*W_alpha_UV.args).doit(), 
    dUV_ellip_sqrd.rhs.subs(*W_alpha_UV.args).expand().collect(W(z),factor)
  )
alpha_no_sqrd = Eq(alpha, solve(dW_alpha.rhs.coeff(W(z),2), alpha)[0])
dW_elliptic = Eq(dW_alpha.lhs, dW_alpha.rhs.subs(*alpha_no_sqrd.args).expand().collect(W(z)))

W_UV_ellptic = W_alpha_UV.subs(*alpha_no_sqrd.args)

g2_ellip = Eq(g2, -dW_elliptic.rhs.coeff(W(z)))
g3_ellip = Eq(g3, -dW_elliptic.rhs.coeff(W(z),0))



ln_dU_ellip = Eq(dU_ellip_b.lhs/U(z), sum(t/U(z) for t in dU_ellip_b.rhs.args))
ln_dUV_ellip = Eq(dUV_ellip.lhs/(U(z)*V(z)) - (dUV_ellip.rhs/(U(z)*V(z))).expand().collect(V(z)**5, factor),0)

ham_sub_for_ln_dUV_ellip = Eq(sum((5*t/(U(z)*V(z))).factor() for t in K_UV_ellip_b2.lhs.args) - 
   sum((5*t/(U(z)*V(z))).factor() for t in K_UV_ellip_b2.rhs.args),
  0
  )

ln_dUV_ellip_b = Eq((ln_dUV_ellip.lhs - ham_sub_for_ln_dUV_ellip.lhs)/2, 
                    (ln_dUV_ellip.rhs - ham_sub_for_ln_dUV_ellip.rhs)/2)

ln_dU_ellip_b = Eq(ln_dU_ellip.lhs + ln_dUV_ellip_b.rhs, ln_dU_ellip.rhs + ln_dUV_ellip_b.lhs)
ln_dU_ellip_c = Eq(ln_dU_ellip_b.lhs, 
                   sum(t.factor().doit() for t in ln_dU_ellip_b.rhs.subs(*W_UV_ellptic.args).args)
                  )
__terms_for_simp = sum(t for t in ln_dU_ellip_c.rhs.args if 
    ((x in t.subs([(diff(W(z),z),y),(W(z),x)]).free_symbols) and 
     (y not in t.subs([(diff(W(z),z),y),(W(z),x)]).free_symbols)
    ))
ln_dU_ellip_c = ln_dU_ellip_c.subs(__terms_for_simp, __terms_for_simp.simplify())
ln_dU_ellip_c = Eq(ln_dU_ellip_c.lhs, ln_dU_ellip_c.rhs.collect((3*X**2+Y**3)/(10*X*Y), factor))

__denom_term_Wb = (12*X**2*Y**2)
pw_y_XY = Eq(pw(y,g2,g3), W(z) - (fraction(ln_dU_ellip_c.rhs.args[1])[1]/__denom_term_Wb).expand())
pwp_y_XY_choice = Eq(pwp(y,g2,g3),X)
pwp_sqrd_y_XY = Eq(dW_elliptic.lhs.subs(diff(W(z),z),pwp(y,g2,g3)), 
   dW_elliptic.rhs.subs(W(z),pw(y,g2,g3)).subs(*pw_y_XY.args).simplify())

pwp_y_XY_choice_check = ((pwp_y_XY_choice.lhs**2 - pwp_sqrd_y_XY.lhs == 0) and 
                         (pwp_y_XY_choice.rhs**2 - pwp_sqrd_y_XY.rhs == 0))
pw_y_XY_W = Eq(((W(z) - pw_y_XY.rhs)*__denom_term_Wb).expand(), (W(z) - pw_y_XY.lhs)*__denom_term_Wb)

W_UV_ellptic_b = W_UV_ellptic.subs([(pw_y_XY.rhs, pw_y_XY.lhs), (W(z), pw(z-z0,g2,g3))])

ln_dU_ellip_d = (ln_dU_ellip_c.subs(*pw_y_XY_W.args)
 .subs([(pwp_y_XY_choice.rhs, pwp_y_XY_choice.lhs),(diff(W(z),z),pwp(z-z0,g2,g3)), (W(z), pw(z-z0,g2,g3))])
)



pw_z_y_to_zw = (pw_to_zw_identity
 .subs([(x,z),(y,-y)])
 .subs([(pwp(-y,g2,g3),-pwp(y,g2,g3)), (pw(-y,g2,g3),pw(y,g2,g3)), (zw(-y,g2,g3),-zw(y,g2,g3))])
).subs(z,z-z0)
ln_dU_ellip_e = ln_dU_ellip_d.subs(*pw_z_y_to_zw.args)

zw_z_y_subs = [
    (Derivative(ln(U(z)),z).doit(), Derivative(ln(U(z)),z)),
    (d_log_sigma_zw.subs(z,z-y).lhs.subs(*d_log_sigma_zw.subs(z,z-y).args).subs(y,z0), 
     Derivative(ln(sigma(z-y,g2,g3)),z).subs(y,z0)),
    (d_log_sigma_zw.subs(z,z-y).lhs.subs(*d_log_sigma_zw.subs(z,z-y).args).subs(y,y+z0), 
     Derivative(ln(sigma(z-y,g2,g3)),z).subs(y,y+z0))
]
ln_dU_ellip_f = ln_dU_ellip_e.subs(zw_z_y_subs)

Uz_sol_ellip = Eq(U(z), 
                  solve(Eq(integrate(ln_dU_ellip_f.lhs,z), 
                           integrate(ln_dU_ellip_f.rhs,z) + c), U(z))[0]
                 )

UV_sigma_z_y = (sigma_p_identity
               .subs([(x,z),(y,-y)])
               .subs(pw(-y,g2,g3),pw(y,g2,g3))
               .subs(z,z-z0)
               .subs(W_UV_ellptic_b.rhs, W_UV_ellptic_b.lhs)
              )

Vz_sol_ellip = Eq(UV_sigma_z_y.lhs/Uz_sol_ellip.lhs, 
   (UV_sigma_z_y.rhs/Uz_sol_ellip.rhs).subs(sigma(-y,g2,g3), -sigma(y,g2,g3)))


dU_ellip
dV_ellip
dU_ellip_b
dV_ellip_b
dUV_ellip
dUV_ellip_sqrd
W_UV_ellptic
dW_elliptic
g2_ellip
g3_ellip
pw_y_XY
pwp_y_XY_choice
print('y existence check:',pwp_y_XY_choice_check)
W_UV_ellptic_b

ln_dU_ellip
ln_dU_ellip_b
ln_dU_ellip_c
ln_dU_ellip_d
ln_dU_ellip_e
ln_dU_ellip_f
Uz_sol_ellip
Vz_sol_ellip

Eq(Derivative(U(z), z), 2*X**5*U(z)/(5*Y*(-X**2 + Y**2*U(z)*V(z))**2) + Y**4*U(z)**6/(5*(-X**2 + Y**2*U(z)*V(z))**2) + Y**4*U(z)*V(z)**5/(5*(-X**2 + Y**2*U(z)*V(z))**2) - Y**2*V(z)**4/(-X**2 + Y**2*U(z)*V(z)) - (3*X**2 + Y**3)*U(z)/(5*X*Y))

Eq(Derivative(V(z), z), -2*X**5*V(z)/(5*Y*(-X**2 + Y**2*U(z)*V(z))**2) - Y**4*U(z)**5*V(z)/(5*(-X**2 + Y**2*U(z)*V(z))**2) - Y**4*V(z)**6/(5*(-X**2 + Y**2*U(z)*V(z))**2) + Y**2*U(z)**4/(-X**2 + Y**2*U(z)*V(z)) + (3*X**2 + Y**3)*V(z)/(5*X*Y))

Eq(Derivative(U(z), z), -X**3*U(z)/(Y*(-X**2 + Y**2*U(z)*V(z))) - Y**2*V(z)**4/(-X**2 + Y**2*U(z)*V(z)) - 2*(3*X**2 + Y**3)*U(z)/(5*X*Y))

Eq(Derivative(V(z), z), X**3*V(z)/(Y*(-X**2 + Y**2*U(z)*V(z))) + Y**2*U(z)**4/(-X**2 + Y**2*U(z)*V(z)) + 2*(3*X**2 + Y**3)*V(z)/(5*X*Y))

Eq(Derivative(U(z)*V(z), z), Y**2*(U(z)**5 - V(z)**5)/(-X**2 + Y**2*U(z)*V(z)))

...

Eq(Derivative(U(z), z)/U(z), (pwp(y, g2, g3) + pwp(z - z0, g2, g3))/(2*(-pw(y, g2, g3) + pw(z - z0, g2, g3))) + (Y**3 + 3*pwp(y, g2, g3)**2)/(10*Y*pwp(y, g2, g3)))

Eq(Derivative(U(z), z)/U(z), zw(y, g2, g3) - zw(z - z0, g2, g3) + zw(-y + z - z0, g2, g3) + (Y**3 + 3*pwp(y, g2, g3)**2)/(10*Y*pwp(y, g2, g3)))

Eq(Derivative(log(U(z)), z), zw(y, g2, g3) - Derivative(log(sigma(z - z0, g2, g3)), z) + Derivative(log(sigma(-y + z - z0, g2, g3)), z) + (Y**3 + 3*pwp(y, g2, g3)**2)/(10*Y*pwp(y, g2, g3)))

Eq(U(z), sigma(-y + z - z0, g2, g3)*exp(Y**2*z/(10*pwp(y, g2, g3)) + z*zw(y, g2, g3) + c + 3*z*pwp(y, g2, g3)/(10*Y))/sigma(z - z0, g2, g3))

Eq(V(z), sigma(y + z - z0, g2, g3)*exp(-Y**2*z/(10*pwp(y, g2, g3)) - z*zw(y, g2, g3) - c - 3*z*pwp(y, g2, g3)/(10*Y))/(sigma(y, g2, g3)**2*sigma(z - z0, g2, g3)))

The constant of integration $c$ can be found using the fact that $U^5$ is a doubly periodic function as shown by the Hamiltonian. There is some ambiguity with regards to which 5th root of unity to pick but we wont dwell on that for now.

In [43]:
quasi_period_z_y = quasi_period.subs([(m,0),(n,1)]).subs(z,z - y - z0).simplify()
Uz_sol_ellip_2_omega1 = (Uz_sol_ellip
                         .subs(z,z+2*omega1)
                         .subs([quasi_period_z_y.args, quasi_period_z_y.subs(y,0).args])
                         .expand().simplify())
U5_periodic = Eq(U(z)**5/U(2*omega1 + z)**5, 1)

U5_periodic_cond_eta1 = Eq(Uz_sol_ellip.lhs**5/Uz_sol_ellip_2_omega1.lhs**5,
   (Uz_sol_ellip.rhs**5/Uz_sol_ellip_2_omega1.rhs**5).simplify()
  ).subs(*U5_periodic.args)

y_UR_periodic = Eq(2*I*pi*n, U5_periodic_cond_eta1.rhs.args[0])

y_solve_UR_periodic = Eq(zw(y,g2,g3), solve(y_UR_periodic,zw(y,g2,g3))[0])

Uz_sol_ellip_eta1 = Uz_sol_ellip.subs(*y_solve_UR_periodic.args).simplify()
Vz_sol_ellip_eta1 = Vz_sol_ellip.subs(*y_solve_UR_periodic.args).simplify()

c_cond = Eq(((Uz_sol_ellip_eta1.rhs.subs(z,z+z0)**5 - Vz_sol_ellip_eta1.rhs.subs(z,-z+z0)**5)
 .subs([(sigma(-z,g2,g3), -sigma(z,g2,g3)), (sigma(y-z,g2,g3), -sigma(-y+z,g2,g3))]).expand().simplify().subs(n,0)
).args[3].expand(),0)
c_cond_b = Eq(exp(10*c), solve(c_cond,exp(10*c))[0])
exp_c_sol = Eq(exp(c), exp(-eta1*y*z0/omega1)/sigma(y,g2,g3)*exp(2*I*pi*k/10))

Uz_sol_ellip_eta1_z_z0 = Uz_sol_ellip_eta1.expand().subs(*exp_c_sol.args).subs([(k,0),(n,0)]).expand().simplify()
Vz_sol_ellip_eta1_z_z0 = Vz_sol_ellip_eta1.expand().subs(*exp_c_sol.args).subs([(k,0),(n,0)]).expand().simplify()


U5_periodic
U5_periodic_cond_eta1
y_UR_periodic
y_solve_UR_periodic
Uz_sol_ellip_eta1
Vz_sol_ellip_eta1

c_cond
c_cond_b
exp_c_sol
Uz_sol_ellip_eta1_z_z0
Vz_sol_ellip_eta1_z_z0

Eq(U(z)**5/U(2*omega1 + z)**5, 1)

Eq(1, exp(-Y**2*omega1/pwp(y, g2, g3) + 10*eta1*y - 10*omega1*zw(y, g2, g3) - 3*omega1*pwp(y, g2, g3)/Y))

Eq(2*I*pi*n, -Y**2*omega1/pwp(y, g2, g3) + 10*eta1*y - 10*omega1*zw(y, g2, g3) - 3*omega1*pwp(y, g2, g3)/Y)

Eq(zw(y, g2, g3), -Y**2/(10*pwp(y, g2, g3)) + eta1*y/omega1 - I*pi*n/(5*omega1) - 3*pwp(y, g2, g3)/(10*Y))

Eq(U(z), sigma(-y + z - z0, g2, g3)*exp(eta1*y*z/omega1 - I*pi*n*z/(5*omega1) + c)/sigma(z - z0, g2, g3))

...

Eq(sigma(y, g2, g3)**10*exp(10*eta1*y*z0/omega1)*exp(10*c) - 1, 0)

Eq(exp(10*c), exp(-10*eta1*y*z0/omega1)/sigma(y, g2, g3)**10)

Eq(exp(c), exp(I*pi*k/5)*exp(-eta1*y*z0/omega1)/sigma(y, g2, g3))

Eq(U(z), sigma(-y + z - z0, g2, g3)*exp(eta1*y*(z - z0)/omega1)/(sigma(y, g2, g3)*sigma(z - z0, g2, g3)))

Eq(V(z), sigma(y + z - z0, g2, g3)*exp(eta1*y*(-z + z0)/omega1)/(sigma(y, g2, g3)*sigma(z - z0, g2, g3)))

Furthermore, via applications of the addition formula it is possible to show that the point $4y$ is congruent to $-y$, and therefore, it can be expressed in terms of a period $\omega_1$:

In [44]:
pw4y_XY = Eq(p_duplication_c.lhs.subs(z,2*y),
   (p_duplication_c.rhs
     .subs(z,2*y)
     .subs(*p_duplication_c.subs(z,y).args)
     .subs(*pw_y_XY.args)
     .subs([g2_ellip.args, g3_ellip.args])
    ).simplify()
  )

pw4_subd_y = p_duplication_c.subs(z,2*y).subs(*p_duplication_c.subs(z,y).args)

pwp4y_XY = Eq(diff(pw4_subd_y.lhs, y), diff(pw4_subd_y.rhs, y).doit()
    .subs([(diff(pw(y,g2,g3),y),X), pw_y_XY.args, g2_ellip.args, g3_ellip.args])
    .simplify())
pwp4y_XY = pwp4y_XY.subs(pwp4y_XY.lhs, 4*pwp(4*y,g2,g3))
pwp4y_XY = Eq(pwp4y_XY.lhs/4, pwp4y_XY.rhs/4)

pw2y_XYa = p_duplication_c.subs(z,y)
pw2y_XY = Eq(pw2y_XYa.lhs, pw2y_XYa.rhs.subs([pw_y_XY.args, g2_ellip.args, g3_ellip.args]).simplify())
pwp2y_XY = Eq(diff(pw2y_XYa.lhs, y), diff(pw2y_XYa.rhs, y).doit()
    .subs([(diff(pw(y,g2,g3),y),X), pw_y_XY.args, g2_ellip.args, g3_ellip.args])
    .simplify())
pwp2y_XY = pwp2y_XY.subs(pwp2y_XY.lhs, 2*pwp(2*y,g2,g3))
pwp2y_XY = Eq(pwp2y_XY.lhs/2, pwp2y_XY.rhs/2)

y_congruent = Eq(4*y, 2*omega1 - y)
y_ham = Eq(y, 2*omega1/5)

Uz_sol_ellip_eta1_z_z0_ham = Uz_sol_ellip_eta1_z_z0.subs(*y_ham.args)
Vz_sol_ellip_eta1_z_z0_hma = Vz_sol_ellip_eta1_z_z0.subs(*y_ham.args)


p_duplication_c.subs(z,2*y)
p_duplication_c.subs(z,y)
g2_ellip
g3_ellip
pw_y_XY
pwp_y_XY_choice
pw2y_XY
pwp2y_XY
pw4y_XY
pwp4y_XY
y_congruent
y_ham
Uz_sol_ellip_eta1_z_z0_ham
Vz_sol_ellip_eta1_z_z0_hma

Eq(pw(4*y, g2, g3), (-g2/2 + 6*pw(2*y, g2, g3)**2)**2/(4*(-g2*pw(2*y, g2, g3) - g3 + 4*pw(2*y, g2, g3)**3)) - 2*pw(2*y, g2, g3))

Eq(pw(2*y, g2, g3), (-g2/2 + 6*pw(y, g2, g3)**2)**2/(4*(-g2*pw(y, g2, g3) - g3 + 4*pw(y, g2, g3)**3)) - 2*pw(y, g2, g3))

Eq(g2, X**4/(12*Y**4) - X**2/Y + 7*Y**2/6 + Y**5/X**2 + Y**8/(12*X**4))

Eq(g3, -X**6/(216*Y**6) + X**4/(12*Y**3) - 25*X**2/72 - 25*Y**6/(72*X**2) - Y**9/(12*X**4) - Y**12/(216*X**6))

Eq(pw(y, g2, g3), X**2/(12*Y**2) + Y/2 + Y**4/(12*X**2))

...

Eq(pwp(4*y, g2, g3), -X)

Eq(4*y, 2*omega1 - y)

Eq(y, 2*omega1/5)

Eq(U(z), sigma(-2*omega1/5 + z - z0, g2, g3)*exp(2*eta1*(z - z0)/5)/(sigma(2*omega1/5, g2, g3)*sigma(z - z0, g2, g3)))

Eq(V(z), sigma(2*omega1/5 + z - z0, g2, g3)*exp(2*eta1*(-z + z0)/5)/(sigma(2*omega1/5, g2, g3)*sigma(z - z0, g2, g3)))

#### Rescaling the fields to reduce the parameters

In [45]:
XY_alph_bet_sols = solve([Eq(X**2,alpha**2*Y**2), Eq(X, 2*(Y**3+3*Y**2*alpha**2)/5/beta/Y).expand()],[X,Y])

X_alph_bet = Eq(X, XY_alph_bet_sols[1][0].expand())
Y_alph_bet = Eq(Y, XY_alph_bet_sols[1][1].expand())

g2_alph_bet = g2_ellip.subs([X_alph_bet.args, Y_alph_bet.args]).simplify()
g3_alph_bet = g3_ellip.subs([X_alph_bet.args, Y_alph_bet.args]).simplify()

Uusub = Eq(U(z),alpha*u(z))
Vvsub = Eq(V(z),alpha*v(z))

du_alpha_beta = Eq(dU_ellip_b.lhs.subs([Uusub.args, Vvsub.args]).doit()/alpha, 
    sum(t.factor()/alpha for t in 
    dU_ellip_b.rhs.subs([X_alph_bet.args, Y_alph_bet.args]).subs([Uusub.args, Vvsub.args]).args)
)
dv_alpha_beta = Eq(dV_ellip_b.lhs.subs([Uusub.args, Vvsub.args]).doit()/alpha, 
    sum(t.factor()/alpha for t in 
    dV_ellip_b.rhs.subs([X_alph_bet.args, Y_alph_bet.args]).subs([Uusub.args, Vvsub.args]).args)
)
Eq(alpha, solve([X_alph_bet,Y_alph_bet], [alpha, beta])[0][0])
Eq(beta, solve([X_alph_bet,Y_alph_bet], [alpha, beta])[0][1])

X_alph_bet
Y_alph_bet
g2_alph_bet
g3_alph_bet
Uusub
Vvsub
du_alpha_beta
dv_alpha_beta

uv_ham_beta_over_alpha = (Eq(UV_ham_XY_poly.lhs/alpha**5, UV_ham_XY_poly.rhs/alpha**5)
.subs([X_alph_bet.args, Y_alph_bet.args, Uusub.args, Vvsub.args]).simplify().expand())
uv_ham_beta_over_alpha

diff(uv_ham_beta_over_alpha.rhs,z).subs([du_alpha_beta.args, dv_alpha_beta.args]).simplify()

Eq(alpha, -X/Y)

Eq(beta, 2*(3*X**2 + Y**3)/(5*X*Y))

Eq(X, 3*alpha**3 + 5*alpha**2*beta/2)

Eq(Y, -3*alpha**2 - 5*alpha*beta/2)

Eq(g2, -20*alpha**4/3 - 25*alpha**3*beta - 125*alpha**2*beta**2/6 + 625*beta**4/192)

...

Eq(V(z), alpha*v(z))

Eq(Derivative(u(z), z), alpha*u(z)/(u(z)*v(z) - 1) - alpha*v(z)**4/(u(z)*v(z) - 1) - u(z)*beta)

Eq(Derivative(v(z), z), alpha*u(z)**4/(u(z)*v(z) - 1) - alpha*v(z)/(u(z)*v(z) - 1) + v(z)*beta)

Eq(3 + 5*beta/(2*alpha), -u(z)**5 + 5*u(z)*v(z) - v(z)**5 - 5*u(z)**2*v(z)**2*beta/(2*alpha) + 5*u(z)*v(z)*beta/alpha)

0

#### Dixon initial conditions

Here we consider $u(0)=0$, $v(0)=1$ but we may also want to consider initial conditions which make $g_2=0$. In the original third order Dixon system those conditions were simultaneously true but that is not the case here.

In [46]:
alpha_1_dix = Eq(alpha,1)
u0_0_dix = Eq(u(0),0)
v0_0_dix = Eq(v(0),1)

beta_dix = Eq(beta, solve(uv_ham_beta_over_alpha.subs(z,0).subs([u0_0_dix.args, v0_0_dix.args]),beta)[0])
uv_ham_dix = uv_ham_beta_over_alpha.subs(*beta_dix.args)
uv_ham_dix = Eq(-uv_ham_dix.lhs, -uv_ham_dix.rhs)

g2_dix = g2_alph_bet.subs(*beta_dix.args).subs(*alpha_1_dix.args)
g3_dix = g3_alph_bet.subs(*beta_dix.args).subs(*alpha_1_dix.args)

du_dix = du_alpha_beta.subs(*beta_dix.args).subs(*alpha_1_dix.args)
dv_dix = dv_alpha_beta.subs(*beta_dix.args).subs(*alpha_1_dix.args)

uz_dix = (
    Uz_sol_ellip_eta1_z_z0_ham.subs([(U(z),u(z)), g2_dix.args, g3_dix.args])
    .subs(z0,-2*omega1/5).expand()
    .subs(exp(4*eta1*omega1/25), 
          1/(sigma(2*omega1/5, Rational(4,3), -Rational(19,27))**2/
             sigma(4*omega1/5, Rational(4,3), -Rational(19,27))))
)
vz_dix = (Vz_sol_ellip_eta1_z_z0_hma.subs([(V(z),v(z)), g2_dix.args, g3_dix.args])
          .subs(z0,-2*omega1/5).expand()
          .subs(exp(-4*eta1*omega1/25), 
                sigma(2*omega1/5, Rational(4,3), -Rational(19,27))**2/
                sigma(4*omega1/5, Rational(4,3), -Rational(19,27)))
         )

u0_0_dix
v0_0_dix
beta_dix
uv_ham_dix
g2_dix
g3_dix
du_dix
dv_dix
du_dix.subs(z,0).subs([u0_0_dix.args, v0_0_dix.args])
dv_dix.subs(z,0).subs([u0_0_dix.args, v0_0_dix.args])

uz_dix
vz_dix

Eq(u(0), 0)

Eq(v(0), 1)

Eq(beta, -8*alpha/5)

Eq(1, u(z)**5 - 4*u(z)**2*v(z)**2 + 3*u(z)*v(z) + v(z)**5)

Eq(g2, 4/3)

...

Eq(Derivative(v(z), z), -8*v(z)/5 + u(z)**4/(u(z)*v(z) - 1) - v(z)/(u(z)*v(z) - 1))

Eq(Subs(Derivative(u(z), z), z, 0), 1)

Eq(Subs(Derivative(v(z), z), z, 0), -3/5)

Eq(u(z), sigma(4*omega1/5, 4/3, -19/27)*sigma(z, 4/3, -19/27)*exp(2*eta1*z/5)/(sigma(2*omega1/5, 4/3, -19/27)**3*sigma(2*omega1/5 + z, 4/3, -19/27)))

Eq(v(z), sigma(2*omega1/5, 4/3, -19/27)*sigma(4*omega1/5 + z, 4/3, -19/27)*exp(-2*eta1*z/5)/(sigma(4*omega1/5, 4/3, -19/27)*sigma(2*omega1/5 + z, 4/3, -19/27)))

### General case with arbitrary valued p

The particular case considered up until this point i.e. the one that fixed the constant to $3 + 5\beta/2\alpha$, is a special case that enabled the factorisation of a polynomial that cancelled the denominator to leave a cubic. When that condition is relaxed to instead consider arbitrary constant $p$, it is neccessary to rethink the polynomials involved to enable the same trick.

In [47]:
gen_conserved_p = Eq(p, uv_ham_beta_over_alpha.rhs.subs(alpha,1))
du_gen = du_alpha_beta.subs(alpha,1)
dv_gen = dv_alpha_beta.subs(alpha,1)
duv_gen = Eq(Derivative(u(z)*v(z),z), diff(u(z)*v(z),z).subs([du_gen.args, dv_gen.args]).simplify())

gen_conserved_p
# Conserved check
diff(gen_conserved_p.rhs,z).subs([du_gen.args, dv_gen.args]).simplify() == 0

du_gen
dv_gen
duv_gen

Eq(p, -u(z)**5 - 5*u(z)**2*v(z)**2*beta/2 + 5*u(z)*v(z)*beta + 5*u(z)*v(z) - v(z)**5)

True

Eq(Derivative(u(z), z), -u(z)*beta + u(z)/(u(z)*v(z) - 1) - v(z)**4/(u(z)*v(z) - 1))

Eq(Derivative(v(z), z), v(z)*beta + u(z)**4/(u(z)*v(z) - 1) - v(z)/(u(z)*v(z) - 1))

Eq(Derivative(u(z)*v(z), z), (u(z)**5 - v(z)**5)/(u(z)*v(z) - 1))

For abitrary $p$ this no longer factors:

In [48]:
duv_gen_sqrd = Eq(duv_gen.lhs**2, 
   (duv_gen.rhs**2 - 
   ((gen_conserved_p.lhs - p + u(z)**5 + v(z)**5)**2 - (gen_conserved_p.rhs - p + u(z)**5 + v(z)**5)**2) /
   (1 - u(z)*v(z))**2).expand().factor()
  )
duv_gen_sqrd

Eq(Derivative(u(z)*v(z), z)**2, (-16*u(z)**5*v(z)**5 + 25*u(z)**4*v(z)**4*beta**2 - 100*u(z)**3*v(z)**3*beta**2 - 100*u(z)**3*v(z)**3*beta + 100*u(z)**2*v(z)**2*beta**2 + 20*u(z)**2*v(z)**2*beta*p + 200*u(z)**2*v(z)**2*beta + 100*u(z)**2*v(z)**2 - 40*u(z)*v(z)*beta*p - 40*u(z)*v(z)*p + 4*p**2)/(4*(u(z)*v(z) - 1)**2))

but for the particualr value of $p=3 + 5\beta/2$ it does:

In [49]:
Eq(duv_gen_sqrd.lhs, duv_gen_sqrd.rhs.subs(p,3 + 5*beta/2).factor().expand().collect(u(z)*v(z),factor))

Eq(Derivative(u(z)*v(z), z)**2, -(5*beta + 4)*(5*beta + 6)*u(z)*v(z)/2 + (5*beta + 6)**2/4 + (25*beta**2 - 32)*u(z)**2*v(z)**2/4 - 4*u(z)**3*v(z)**3)

## System 2: Pseudo-Hyperelliptic Hamiltonian

In this section we do not renormalise the Hamiltonian to obtain elliptic function solutions as we did in System 1. Instead we use the following polynomial in $U,V$ as the Hamiltonian and we end up with a differential equation for $U(z)V(z)$ that, were it not for the repeated root, would suggest it is hyperelliptic (genus 2). Because of the repeated root I'm refering to this as pseudo-hyperelliptic for now. It may even be the case that because of the repeated root this differential equation is pseudo-elliptic. It may in fact be related to [Carlson's $R_J$ elliptic integral](https://dlmf.nist.gov/19.16) in a similar way that [$\wp$ is related to $R_F$](https://dlmf.nist.gov/19.25).

In [50]:

UV_ham_XY_poly_b = Eq(-UV_ham_XY_poly.lhs - X**3/Y**2 + U(z)**5 + V(z)**5, 
                      -UV_ham_XY_poly.rhs - X**3/Y**2 + U(z)**5 + V(z)**5)
dUgen2 = Eq(diff(U(z),z), diff(UV_ham_XY_poly.rhs, V(z)))
dVgen2 = Eq(diff(V(z),z), -diff(UV_ham_XY_poly.rhs, U(z)))
dUVgen2 = Eq(Derivative(U(z)*V(z),z), diff(U(z)*V(z),z).doit().subs([dUgen2.args, dVgen2.args]).expand())
dUVgen2_sqrd = Eq(dUVgen2.lhs**2, dUVgen2.rhs**2 )
dUVgen2_sqrd_b = Eq(dUVgen2_sqrd.lhs, 
                    (dUVgen2_sqrd.rhs - (5*UV_ham_XY_poly_b.lhs)**2 + (5*UV_ham_XY_poly_b.rhs)**2)
                    .expand().collect(U(z)*V(z), factor)
                   )
dUVgen2_sqrd_c = Eq(dUVgen2_sqrd_b.lhs, 
                    (dUVgen2_sqrd_b.rhs.factor()/(-X**2 +Y**2*U(z)*V(z))**2)
                    .collect(U(z)*V(z), factor)
                    *(-X**2 +Y**2*U(z)*V(z))**2
                   )
dW_UV_XY_hyp = Eq(U(z)*V(z), W(z))
dW_sqrd_hyp = dUVgen2_sqrd_c.subs(U(z),solve(dW_UV_XY_hyp,U(z))[0]).doit()
dW_sqrd_hyp = Eq(dW_sqrd_hyp.lhs,
                 (dW_sqrd_hyp.rhs/(W(z)*Y**2-X**2)**2).expand().collect(W(z),factor) * (W(z)*Y**2-X**2)**2
                )

ln_dUgen2 = Eq(dUgen2.lhs/U(z), dUgen2.rhs/U(z)).expand().subs(*dW_UV_XY_hyp.args)

UV_ham_XY_poly_c = Eq((5*UV_ham_XY_poly_b.lhs/(U(z)*V(z))).expand(), 
                      (5*UV_ham_XY_poly_b.rhs/(U(z)*V(z))).subs(*dW_UV_XY_hyp.args).expand().collect(W(z), factor)
                     )

ln_dUVgen2 = Eq((dUVgen2.rhs/(U(z)*V(z))).expand(), (dUVgen2.lhs/(U(z)*V(z))).subs(*dW_UV_XY_hyp.args))

V4_U = Eq((UV_ham_XY_poly_c.lhs + ln_dUVgen2.lhs)/2, (UV_ham_XY_poly_c.rhs + ln_dUVgen2.rhs)/2)
ln_dUgen2_W = Eq(ln_dUgen2.lhs, ln_dUgen2.rhs.subs(*V4_U.args).expand().collect(W(z), factor))


UV_ham_XY_poly
UV_ham_XY_poly_b
dUgen2
dVgen2
dUVgen2
dUVgen2_sqrd
dUVgen2_sqrd_b
dUVgen2_sqrd_c
dW_UV_XY_hyp
dW_sqrd_hyp

# ln_dUgen2
# UV_ham_XY_poly_c
# ln_dUVgen2
# V4_U
ln_dUgen2_W

Eq(-X**3/Y**2, -(X**3/Y**3 + 2*X)*U(z)*V(z) + U(z)**5 + V(z)**5 + (3*X**2 + Y**3)*U(z)**2*V(z)**2/(X*Y))

Eq(U(z)**5 + V(z)**5, -X**3/Y**2 + (X**3/Y**3 + 2*X)*U(z)*V(z) - (3*X**2 + Y**3)*U(z)**2*V(z)**2/(X*Y))

Eq(Derivative(U(z), z), -(X**3/Y**3 + 2*X)*U(z) + 5*V(z)**4 + 2*(3*X**2 + Y**3)*U(z)**2*V(z)/(X*Y))

Eq(Derivative(V(z), z), (X**3/Y**3 + 2*X)*V(z) - 5*U(z)**4 - 2*(3*X**2 + Y**3)*U(z)*V(z)**2/(X*Y))

Eq(Derivative(U(z)*V(z), z), -5*U(z)**5 + 5*V(z)**5)

...

Eq(Derivative(U(z)*V(z), z)**2, 25*X**6/Y**4 - 50*X**4*(X**2 + 2*Y**3)*U(z)*V(z)/Y**5 + 25*X**2*(X**4 + 10*X**2*Y**3 + 6*Y**6)*U(z)**2*V(z)**2/Y**6 - 100*U(z)**5*V(z)**5 - 50*(X**2 + 2*Y**3)*(3*X**2 + Y**3)*U(z)**3*V(z)**3/Y**4 + 25*(3*X**2 + Y**3)**2*U(z)**4*V(z)**4/(X**2*Y**2))

Eq(Derivative(U(z)*V(z), z)**2, (-X**2 + Y**2*U(z)*V(z))**2*(25*X**4*Y**2 - 100*X**2*Y**2*U(z)**3*V(z)**3 - 50*X**2*Y*(X**2 + Y**3)*U(z)*V(z) + 25*(X**4 + 6*X**2*Y**3 + Y**6)*U(z)**2*V(z)**2)/(X**2*Y**6))

Eq(U(z)*V(z), W(z))

Eq(Derivative(W(z), z)**2, (-X**2 + Y**2*W(z))**2*(25*X**2/Y**4 - 100*W(z)**3/Y**4 - 50*(X**2 + Y**3)*W(z)/Y**5 + 25*(X**4 + 6*X**2*Y**3 + Y**6)*W(z)**2/(X**2*Y**6)))

Eq(Derivative(U(z), z)/U(z), -5*X**3/(2*Y**2*W(z)) + 3*X*(X**2 + 2*Y**3)/(2*Y**3) + Derivative(W(z), z)/(2*W(z)) - (3*X**2 + Y**3)*W(z)/(2*X*Y))

As well as observing the repeated root, it is noteworthy that that the other cubic part takes the identical form as the cubic in the Weierstrass P differential equation.

In [51]:
dW_sqrd_hyp_b = Eq(Derivative(W(z), z)**2,
 (-X**2/Y**2 + W(z))**2*
   25*((25*X**2/Y**4 - 100*W(z)**3/Y**4 - 50*(X**2 + Y**3)*W(z)/Y**5 + 
    25*(X**4 + 6*X**2*Y**3 + Y**6)*W(z)**2/(X**2*Y**6))*(Y**4)/25).expand().collect(W(z)))

W_f_XY_sub = Eq(W(z), f(z) + X**2/(12*Y**2) + Y/2 + Y**4/(12*X**2))
dW_sqrd_hyp_c = dW_sqrd_hyp_b.subs(*W_f_XY_sub.args).doit()

dW_sqrd_hyp_c = Eq(Derivative(f(z), z)**2,
 (X**2 + (-2*X**2/Y - 2*Y**2)*(X**2/(12*Y**2) + Y/2 + f(z) + Y**4/(12*X**2)) + 
     (X**2/Y**2 + 6*Y + Y**4/X**2)*(X**2/(12*Y**2) + Y/2 + f(z) + Y**4/(12*X**2))**2 - 
     4*(X**2/(12*Y**2) + Y/2 + f(z) + Y**4/(12*X**2))**3).expand().collect(f(z))
                   *(-11*X**2/(12*Y**2) + Y/2 + f(z) + Y**4/(12*X**2)).expand()**2*25
                  )
f_g2_g3_subs = [(-g2XY.rhs, -g2XY.lhs), (-g3XY.rhs, -g3XY.lhs), pwx_2_4_XY_b.args]
df_hyp = dW_sqrd_hyp_c.subs(f_g2_g3_subs)
pwx = Eq(
    pw(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2, 
   pw(x,g2,g3)
)
df_hyp_b = df_hyp.subs(*pwx.args)

dW_sqrd_hyp_b
W_f_XY_sub
dW_sqrd_hyp_c
df_hyp
pwx
df_hyp_b

Eq(Derivative(W(z), z)**2, 25*(-X**2/Y**2 + W(z))**2*(X**2 + (-2*X**2/Y - 2*Y**2)*W(z) + (X**2/Y**2 + 6*Y + Y**4/X**2)*W(z)**2 - 4*W(z)**3))

Eq(W(z), X**2/(12*Y**2) + Y/2 + f(z) + Y**4/(12*X**2))

Eq(Derivative(f(z), z)**2, 25*(-11*X**2/(12*Y**2) + Y/2 + f(z) + Y**4/(12*X**2))**2*(X**6/(216*Y**6) - X**4/(12*Y**3) + 25*X**2/72 + (X**4/(12*Y**4) - X**2/Y + 7*Y**2/6 + Y**5/X**2 + Y**8/(12*X**4))*f(z) - 4*f(z)**3 + 25*Y**6/(72*X**2) + Y**9/(12*X**4) + Y**12/(216*X**6)))

Eq(Derivative(f(z), z)**2, 25*(g2*f(z) - g3 - 4*f(z)**3)*(f(z) + pw(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2)**2)

Eq(pw(2*omega1/5, g2, g3) - pwp(2*omega1/5, g2, g3)**2/(pw(2*omega1/5, g2, g3) - pw(4*omega1/5, g2, g3))**2, pw(x, g2, g3))

Eq(Derivative(f(z), z)**2, 25*(f(z) + pw(x, g2, g3))**2*(g2*f(z) - g3 - 4*f(z)**3))

By setting $f$ to be a function of some other function that meets a certain criteria, then by the chain rule it can be shown that there is a connection between $f$ and the Weierstrass P function. The other function is the inverse of the Kronecker theta phase.

In [52]:
fh_rho_sub = Eq(f(z),-h(rho(z)))

dh_rhoz = df_hyp.subs(*pwx.args).subs(*fh_rho_sub.args).doit()
d_rhoz_h = Eq(diff(rho(z),z), 5*(h(rho(z)) - pw(x,g2,g3)))
dh_rhoz_b = dh_rhoz.subs(*d_rhoz_h.args)
dh_rhoz_c = Eq(
    dh_rhoz_b.lhs/(dh_rhoz_b.lhs.subs(diff(h(rho(z)),rho(z)),1)), 
    dh_rhoz_b.rhs/(dh_rhoz_b.lhs.subs(diff(h(rho(z)),rho(z)),1))
).subs(rho(z), t).simplify()
h_pw = Eq(h(t), pw(t-t0,g2,g3))
fp_rho_sub = Eq(f(z),-pw(rho(z)-t0,g2,g3))

integral_rhoz_t = Eq(Integral(1/(pw(t-t0,g2,g3) - pw(x,g2,g3)),(t,rho(z),c)), z + C)
integral_rhoz_t_b = Eq(Integral(pwp(x,g2,g3)/(-pw(y,g2,g3) + pw(x,g2,g3)),(y,rho(z) - t0, c - t0)), 
                     -pwp(x,g2,g3)*z  - pwp(x,g2,g3)*C)
integral_rhoz_t_c = Eq(Integral(
    (pwp(x,g2,g3)/(-pw(y,g2,g3) + pw(x,g2,g3))).subs(*pwp_x_y_log_sigma_diff_c.args),(y,rho(z) - t0,c - t0)), 
                     -pwp(x,g2,g3)*z + -pwp(x,g2,g3)*C)
integral_rhoz_t_d = Eq(
    (log(sigma(x + y, g2, g3)*exp(-2*y*zw(x, g2, g3))/sigma(-x + y, g2, g3))).subs(y, -t0 + rho(z))
    -(log(sigma(x + y, g2, g3)*exp(-2*y*zw(x, g2, g3))/sigma(-x + y, g2, g3))).subs(y, -t0 + c),
    C*pwp(x, g2, g3) + z*pwp(x, g2, g3)
)
mu_const = Eq(mu,
    (log(sigma(x + y, g2, g3)*exp(-2*y*zw(x, g2, g3))/sigma(-x + y, g2, g3))).subs(y, -t0 + c) + C*pwp(x, g2, g3)
)
rho_r = Eq(rho(z), r(z) + t0)
integral_r = Eq(
    (log(sigma(x + y, g2, g3)*exp(-2*y*zw(x, g2, g3))/sigma(-x + y, g2, g3)))
    .subs(y, -t0 + rho(z)).subs(*rho_r.args),
    mu
    + z*pwp(x, g2, g3)
)

fr_sub = fp_rho_sub.subs(*rho_r.args)
integral_rhoz_t_d = integral_rhoz_t_d
d_rz_h = d_rhoz_h.subs(*h_pw.subs(t,rho(z)).args).subs(*rho_r.args).doit()

fh_rho_sub
dh_rhoz
d_rhoz_h
dh_rhoz_c
h_pw
fp_rho_sub

integral_rhoz_t
integral_rhoz_t_b
pwp_x_y_log_sigma_diff_c
integral_rhoz_t_c
integral_rhoz_t_d
rho_r
mu_const

fr_sub
integral_r
d_rz_h

Eq(f(z), -h(rho(z)))

Eq(Derivative(h(rho(z)), rho(z))**2*Derivative(rho(z), z)**2, 25*(-h(rho(z)) + pw(x, g2, g3))**2*(-g2*h(rho(z)) - g3 + 4*h(rho(z))**3))

Eq(Derivative(rho(z), z), 5*h(rho(z)) - 5*pw(x, g2, g3))

Eq(Derivative(h(t), t)**2, -g2*h(t) - g3 + 4*h(t)**3)

Eq(h(t), pw(t - t0, g2, g3))

...

Eq(rho(z), t0 + r(z))

Eq(mu, C*pwp(x, g2, g3) + log(sigma(-t0 + x + c, g2, g3)*exp(-2*(-t0 + c)*zw(x, g2, g3))/sigma(-t0 - x + c, g2, g3)))

Eq(f(z), -pw(r(z), g2, g3))

Eq(log(sigma(x + r(z), g2, g3)*exp(-2*r(z)*zw(x, g2, g3))/sigma(-x + r(z), g2, g3)), z*pwp(x, g2, g3) + mu)

Eq(Derivative(r(z), z), -5*pw(x, g2, g3) + 5*pw(r(z), g2, g3))

In [53]:
# https://dlmf.nist.gov/19.16
RJ = Function('RJ')
Rj_carlson = Eq(RJ(x,y,z,p), Rational(3,2)*Integral(1/(t+p)/sqrt((t+x)*(t+y)*(t+z)),(t,0,oo)))
Rj_carlson_b = Eq(RJ(x,y,z,p), Rational(3,2)*Integral(
    1/(t+p - epsilon)/sqrt((t+x - epsilon)*(t+y - epsilon)*(t+z - epsilon)),(t,epsilon,oo)))
Rj_carlson_c = (Rj_carlson_b
                .subs([(x,x+epsilon),(y,y+epsilon),(z,z+epsilon),(p,p+epsilon)])
                .subs([(x,mu[1]),(y,mu[2]),(z,mu[3])])
                .subs([(epsilon,r(z))])
               )
Rj_carlson_c_z = Eq(Rj_carlson_c.lhs, z + c)
Rj_carlson_d_z = Rj_carlson_c.subs(*Rj_carlson_c_z.args)
Rj_carlson_d_z_diff = Eq(diff(Rj_carlson_d_z.lhs,z), diff(Rj_carlson_d_z.rhs,z))
Rj_carlson_d_z_diff_b = Eq(diff(r(z),z)**2, 
                           solve(Eq(Rj_carlson_d_z_diff.lhs**2, Rj_carlson_d_z_diff.rhs**2),
                                 diff(r(z),z)**2)[0].factor()
                          )
rz_chi_sub = Eq(r(z),h(chi(z)))
chi_r = Rj_carlson_d_z_diff_b.subs(*rz_chi_sub.args).doit()
drz_chi_sub = Eq(diff(chi(z),z), (h(chi(z))+p)/3)
h_chi_z = Eq(h(t), pw(t,g2,g3))
mu_omega1 = Eq(mu[1], -pw(omega1,g2,g3))
mu_omega2 = Eq(mu[2], -pw(omega2,g2,g3))
mu_omega3 = Eq(mu[3], -pw(omega3,g2,g3))
p_pw_x = Eq(p, -pw(x,g2,g3))
zZ3 = Eq(z, Z*3)
dh_chi_sub_Z = Eq(Derivative(chi(Z*3),Z), (pw(chi(Z*3),g2,g3) - pw(x,g2,g3)))

carlson_chi_inv = Rj_carlson_c_z.subs([
    (r(z),pw(chi(z),g2,g3)), mu_omega1.args, mu_omega2.args, mu_omega3.args, p_pw_x.args])

dphi_Z5 = Eq(Derivative(phi(Z/5), z), -pw(x, g2, g3) + pw(phi(Z/5), g2, g3))

chi_phi = Eq(chi(3*Z), phi(Z/5)).subs(Z,z/3)

Rj_carlson
Rj_carlson_b
Rj_carlson_c
Rj_carlson_c_z
Rj_carlson_d_z
Rj_carlson_d_z_diff
Rj_carlson_d_z_diff_b
rz_chi_sub
chi_r
drz_chi_sub
h_chi_z
mu_omega1
mu_omega2
mu_omega3
p_pw_x
carlson_chi_inv

dh_chi_sub_Z
dphi_Z5
chi_phi

Eq(RJ(x, y, z, p), 3*Integral(1/(sqrt((t + x)*(t + y)*(t + z))*(t + p)), (t, 0, oo))/2)

Eq(RJ(x, y, z, p), 3*Integral(1/(sqrt((t + x - epsilon)*(t + y - epsilon)*(t + z - epsilon))*(t - epsilon + p)), (t, epsilon, oo))/2)

Eq(RJ(r(z) + mu[1], r(z) + mu[2], r(z) + mu[3], r(z) + p), 3*Integral(1/(sqrt((t + mu[1])*(t + mu[2])*(t + mu[3]))*(t + p)), (t, r(z), oo))/2)

Eq(RJ(r(z) + mu[1], r(z) + mu[2], r(z) + mu[3], r(z) + p), z + c)

Eq(z + c, 3*Integral(1/(sqrt((t + mu[1])*(t + mu[2])*(t + mu[3]))*(t + p)), (t, r(z), oo))/2)

...

Eq(p, -pw(x, g2, g3))

Eq(RJ(-pw(omega1, g2, g3) + pw(chi(z), g2, g3), -pw(omega2, g2, g3) + pw(chi(z), g2, g3), -pw(omega3, g2, g3) + pw(chi(z), g2, g3), -pw(x, g2, g3) + pw(chi(z), g2, g3)), z + c)

Eq(Derivative(chi(3*Z), Z), -pw(x, g2, g3) + pw(chi(3*Z), g2, g3))

Eq(Derivative(phi(Z/5), z), -pw(x, g2, g3) + pw(phi(Z/5), g2, g3))

Eq(chi(z), phi(z/15))

### Rescaling the second system

In [54]:
delta_alpha_beta = Eq(5*beta/2/alpha, delta)
uv__delta_ham = Eq(uv_ham_beta_over_alpha.lhs - 5*beta/2/alpha, 
   (uv_ham_beta_over_alpha.rhs - 5*beta/2/alpha).subs(beta,solve(delta_alpha_beta,beta)[0]).collect(delta,factor)
  )
uv__delta_ham_b = Eq(uv__delta_ham.lhs - 3 + u(z)**5 + v(z)**5, uv__delta_ham.rhs - 3 + u(z)**5 + v(z)**5)

delta_alpha_beta
uv__delta_ham
uv__delta_ham_b

Eq(5*beta/(2*alpha), delta)

Eq(3, -delta*(u(z)*v(z) - 1)**2 - u(z)**5 + 5*u(z)*v(z) - v(z)**5)

Eq(u(z)**5 + v(z)**5, -delta*(u(z)*v(z) - 1)**2 + 5*u(z)*v(z) - 3)

In [55]:
du_delta = Eq(diff(u(z),z), diff(uv__delta_ham.rhs, v(z)))
dv_delta = Eq(diff(v(z),z), -diff(uv__delta_ham.rhs, u(z)))

du_delta
dv_delta

Eq(Derivative(u(z), z), -2*delta*(u(z)*v(z) - 1)*u(z) + 5*u(z) - 5*v(z)**4)

Eq(Derivative(v(z), z), 2*delta*(u(z)*v(z) - 1)*v(z) + 5*u(z)**4 - 5*v(z))

In [56]:
(5*uv__delta_ham.lhs - 5*uv__delta_ham.rhs).expand()

5*delta*u(z)**2*v(z)**2 - 10*delta*u(z)*v(z) + 5*delta + 5*u(z)**5 - 25*u(z)*v(z) + 5*v(z)**5 + 15

In [57]:
Eq(Derivative(ln(u(z)/v(z)),z,z), 
   diff(diff(ln(u(z)/v(z)),z).doit().subs([du_delta.args, dv_delta.args]).simplify(),z)
   .doit().subs([du_delta.args, dv_delta.args]).factor()
  )

Eq(Derivative(log(u(z)/v(z)), (z, 2)), 5*(u(z) - v(z))*(6*delta*u(z)**2*v(z)**2 - 10*delta*u(z)*v(z) + 5*u(z)**5 - 25*u(z)*v(z) + 5*v(z)**5)*(u(z)**4 + u(z)**3*v(z) + u(z)**2*v(z)**2 + u(z)*v(z)**3 + v(z)**4)/(u(z)**2*v(z)**2))

In [58]:
duv_delta = Eq(Derivative(u(z)*v(z),z), diff(u(z)*v(z),z).subs([du_delta.args, dv_delta.args]).simplify())
duv_delta_sqrd = Eq(duv_delta.lhs**2, 
                    (duv_delta.rhs**2).expand() -
                    ((5*uv__delta_ham_b.lhs)**2).expand() + (5*uv__delta_ham_b.rhs)**2
                   )


duv_delta
duv_delta_sqrd

Eq(Derivative(u(z)*v(z), z), 5*u(z)**5 - 5*v(z)**5)

Eq(Derivative(u(z)*v(z), z)**2, (-5*delta*(u(z)*v(z) - 1)**2 + 25*u(z)*v(z) - 15)**2 - 100*u(z)**5*v(z)**5)

In [59]:
duv_delta_sqrd.subs(u(z),(W(z))/v(z)).rhs.expand().factor()

-25*(W(z) - 1)**2*(-delta**2*W(z)**2 + 2*delta**2*W(z) - delta**2 + 10*delta*W(z) - 6*delta + 4*W(z)**3 + 8*W(z)**2 + 12*W(z) - 9)

In [60]:
dW_delta = duv_delta_sqrd.subs(u(z),(W(z))/v(z))
dW_delta

Eq(Derivative(W(z), z)**2, (-5*delta*(W(z) - 1)**2 + 25*W(z) - 15)**2 - 100*W(z)**5)

In [61]:
v4_duv = Eq(0, 
   duv_delta.lhs/u(z)/v(z)/2 -
   ((duv_delta.rhs - 5* uv__delta_ham_b.lhs + 5*uv__delta_ham_b.rhs)/u(z)/v(z)/2)
   .expand().collect([delta,v(z)**4], factor)
  )
v4_duv

Eq(0, 5*delta*(u(z)*v(z) - 1)**2/(2*u(z)*v(z)) - 5*(5*u(z)*v(z) - 3)/(2*u(z)*v(z)) + 5*v(z)**4/u(z) + Derivative(u(z)*v(z), z)/(2*u(z)*v(z)))

In [62]:
Eq(du_delta.lhs/u(z) + v4_duv.lhs, 
   apart(
       ((du_delta.rhs/u(z)).expand().collect([v(z)**4,delta], factor) + v4_duv.rhs)
       .subs(v(z),W(z)/u(z)).subs(diff(W(z),z),Y),
       W(z)
  ).subs(Y,diff(W(z),z))
)

Eq(Derivative(u(z), z)/u(z), delta*W(z)/2 - 3*delta + (5*delta + Derivative(W(z), z) + 15)/(2*W(z)) - 15/2)

## On the connection between System 1 and System 2

The two different Hamiltonian systems can be related through a substitution that changes the argument of the function into a function of some other variable.

In [63]:
du_method1 = dUgen2.subs([(U(z),u(z)), (V(z),v(z))])
dv_method1 = dVgen2.subs([(U(z),u(z)), (V(z),v(z))])

Uuf = Eq(U(z),u(f(z)))
Vvf = Eq(V(z),v(f(z)))
df_uf_vf = Eq(diff(f(z),z), -Y**2/(-X**2+Y**2*u(f(z))*v(f(z)))/5)
dUu_fz= (dU_ellip_b
 .subs([Uuf.args, Vvf.args]).doit()
 .subs(*df_uf_vf.args)
 .subs([(f(z),y)]).subs(y,z)
)
dUu_fz_numer, dUu_fz_denom = fraction(dUu_fz.lhs)
dUu_fz = Eq((-dUu_fz.lhs*dUu_fz_denom/Y**2).simplify(), 
            (-dUu_fz.rhs*dUu_fz_denom/Y**2).simplify().expand().collect([u(z),v(z)], factor))
dVv_fz= (dV_ellip_b
 .subs([Uuf.args, Vvf.args]).doit()
 .subs(*df_uf_vf.args)
 .subs([(f(z),y)]).subs(y,z)
)
dVv_fz_numer, dVv_fz_denom = fraction(dVv_fz.lhs)
dVv_fz = Eq((-dVv_fz.lhs*dVv_fz_denom/Y**2).simplify(), 
            (-dVv_fz.rhs*dVv_fz_denom/Y**2).simplify().expand().collect([u(z),v(z)], factor))

Uuf
Vvf
df_uf_vf

dUu_fz
dVv_fz

du_method1
dv_method1
dU_ellip_b
dV_ellip_b

(dUu_fz.lhs - du_method1.lhs).simplify() == 0
(dUu_fz.rhs - du_method1.rhs).simplify() == 0
(dVv_fz.lhs - dv_method1.lhs).simplify() == 0
(dVv_fz.rhs - dv_method1.rhs).simplify() == 0

Eq(U(z), u(f(z)))

Eq(V(z), v(f(z)))

Eq(Derivative(f(z), z), -Y**2/(5*(-X**2 + Y**2*u(f(z))*v(f(z)))))

Eq(Derivative(u(z), z), -X*(X**2 + 2*Y**3)*u(z)/Y**3 + 5*v(z)**4 + 2*(3*X**2 + Y**3)*u(z)**2*v(z)/(X*Y))

Eq(Derivative(v(z), z), X*(X**2 + 2*Y**3)*v(z)/Y**3 - 5*u(z)**4 - 2*(3*X**2 + Y**3)*u(z)*v(z)**2/(X*Y))

...

Eq(Derivative(V(z), z), X**3*V(z)/(Y*(-X**2 + Y**2*U(z)*V(z))) + Y**2*U(z)**4/(-X**2 + Y**2*U(z)*V(z)) + 2*(3*X**2 + Y**3)*V(z)/(5*X*Y))

True

True

True

True